This dataset contains information about candidates, committees, PACs (political action committees), House & Senate campaigns & about transactions, contributions by individuals & committees & operating & independent expenditures for US elections from Jan 1, 2019 to May 2020. The original dataset was prodcured from: https://www.fec.gov/data/browse-data/?tab=bulk-data.

##### Analysis of North Carolina Senate race 2020: Thom Tillis & Cal Cunningham

#### Here I address the key question requested in prompt number three.

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import bs4
import urllib.request

# native python module
import os
import cufflinks


pd.set_option('display.max_columns', 1000)


base_dir = '.../data/data/'
dataset_name = '20192020-FEC/'
for dirname, _, filenames in os.walk(base_dir + '/' + dataset_name):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
state = 'NC'
candidate1 = 'tillis, thom'
candidate2 = 'cunningham, cal'

In [3]:
import bs4
import urllib.request

def read_table(page_url):
    page = urllib.request.urlopen(page_url)
    soup = bs4.BeautifulSoup(page, 'lxml')
    table = soup.find(name='table')
    result = dict()
    for tr in table.findAll('tr'):
        tds = tr.findAll('td')
        
        if len(tds) >= 2:
            result[tds[0].text] = tds[1].text
    return result

In [4]:
def render_human_format(num):
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    # add more suffixes if you need them
    return '%.2f%s' % (num, ['', 'K', 'M', 'G', 'T', 'P'][magnitude])

## Candidate Campaign Finance summaries

In [5]:
all_candidates = pd.read_csv("../data/20192020-FEC/All candidates.csv")
all_candidates.head()

,CAND_ID,CAND_NAME,CAND_ICI,PTY_CD,CAND_PTY_AFFILIATION,TTL_RECEIPTS,TRANS_FROM_AUTH,TTL_DISB,TRANS_TO_AUTH,COH_BOP,COH_COP,CAND_CONTRIB,CAND_LOANS,OTHER_LOANS,CAND_LOAN_REPAY,OTHER_LOAN_REPAY,DEBTS_OWED_BY,TTL_INDIV_CONTRIB,CAND_OFFICE_ST,CAND_OFFICE_DISTRICT,SPEC_ELECTION,PRIM_ELECTION,RUN_ELECTION,GEN_ELECTION,GEN_ELECTION_PRECENT,OTHER_POL_CMTE_CONTRIB,POL_PTY_CONTRIB,CVG_END_DT,INDIV_REFUNDS,CMTE_REFUNDS
0,H8AK00132,"SHEIN, DIMITRI",C,1,DEM,0.00,0.00,367.52,0.0,367.52,0.00,0.00,0.0,0.0,367.52,0.0,0.00,0.00,AK,0,NaN,NaN,NaN,NaN,NaN,0.00,0.0,09/30/2019,0.00,0.0
1,H6AK00045,"YOUNG, DONALD E",I,2,REP,1012401.48,130245.48,325545.51,0.0,116720.12,803576.09,0.00,0.0,0.0,0.00,0.0,0.00,417190.55,AK,0,NaN,NaN,NaN,NaN,NaN,458194.63,0.0,03/31/2020,0.00,2000.0
2,H8AK01031,"NELSON, THOMAS JOHN",C,2,REP,0.00,0.00,466.51,0.0,466.51,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00,AK,0,NaN,NaN,NaN,NaN,NaN,0.00,0.0,03/31/2019,0.00,0.0
3,H8AK00140,"GALVIN, ALYSE",C,3,IND,1358372.81,0.00,403275.92,0.0,6245.09,961341.98,2955.81,0.0,0.0,0.00,0.0,56940.58,1279739.80,AK,0,NaN,NaN,NaN,NaN,NaN,38350.00,0.0,03/31/2020,7836.19,0.0
4,H0AL01097,"AVERHART, JAMES",O,1,DEM,50126.74,0.00,40699.28,0.0,0.00,9427.46,0.00,26845.0,0.0,0.00,0.0,26845.00,23281.74,AL,1,NaN,NaN,NaN,NaN,NaN,0.00,0.0,06/30/2020,0.00,0.0


In [6]:
def receipts_disbursements_clean_up_func(df):
    
    predicate = (df['TRANS_FROM_AUTH'] != 0.0) & (df['TRANS_TO_AUTH'] != 0.0)
    
    df.loc[predicate, 'TTL_RECEIPTS_CORRECTED'] = df['TTL_RECEIPTS'] - df['TRANS_FROM_AUTH']
    df['TTL_RECEIPTS_CORRECTED'] = df['TTL_RECEIPTS_CORRECTED'].fillna(df['TTL_RECEIPTS'])
    
    df.loc[predicate, 'TTL_DISB_CORRECTED'] = df['TTL_DISB'] - df['TRANS_TO_AUTH']
    df['TTL_DISB_CORRECTED'] = df['TTL_DISB_CORRECTED'].fillna(df['TTL_DISB'])
    
    return df

all_candidates = receipts_disbursements_clean_up_func(all_candidates)
all_candidates.head()

,CAND_ID,CAND_NAME,CAND_ICI,PTY_CD,CAND_PTY_AFFILIATION,TTL_RECEIPTS,TRANS_FROM_AUTH,TTL_DISB,TRANS_TO_AUTH,COH_BOP,COH_COP,CAND_CONTRIB,CAND_LOANS,OTHER_LOANS,CAND_LOAN_REPAY,OTHER_LOAN_REPAY,DEBTS_OWED_BY,TTL_INDIV_CONTRIB,CAND_OFFICE_ST,CAND_OFFICE_DISTRICT,SPEC_ELECTION,PRIM_ELECTION,RUN_ELECTION,GEN_ELECTION,GEN_ELECTION_PRECENT,OTHER_POL_CMTE_CONTRIB,POL_PTY_CONTRIB,CVG_END_DT,INDIV_REFUNDS,CMTE_REFUNDS,TTL_RECEIPTS_CORRECTED,TTL_DISB_CORRECTED
0,H8AK00132,"SHEIN, DIMITRI",C,1,DEM,0.00,0.00,367.52,0.0,367.52,0.00,0.00,0.0,0.0,367.52,0.0,0.00,0.00,AK,0,NaN,NaN,NaN,NaN,NaN,0.00,0.0,09/30/2019,0.00,0.0,0.00,367.52
1,H6AK00045,"YOUNG, DONALD E",I,2,REP,1012401.48,130245.48,325545.51,0.0,116720.12,803576.09,0.00,0.0,0.0,0.00,0.0,0.00,417190.55,AK,0,NaN,NaN,NaN,NaN,NaN,458194.63,0.0,03/31/2020,0.00,2000.0,1012401.48,325545.51
2,H8AK01031,"NELSON, THOMAS JOHN",C,2,REP,0.00,0.00,466.51,0.0,466.51,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00,AK,0,NaN,NaN,NaN,NaN,NaN,0.00,0.0,03/31/2019,0.00,0.0,0.00,466.51
3,H8AK00140,"GALVIN, ALYSE",C,3,IND,1358372.81,0.00,403275.92,0.0,6245.09,961341.98,2955.81,0.0,0.0,0.00,0.0,56940.58,1279739.80,AK,0,NaN,NaN,NaN,NaN,NaN,38350.00,0.0,03/31/2020,7836.19,0.0,1358372.81,403275.92
4,H0AL01097,"AVERHART, JAMES",O,1,DEM,50126.74,0.00,40699.28,0.0,0.00,9427.46,0.00,26845.0,0.0,0.00,0.0,26845.00,23281.74,AL,1,NaN,NaN,NaN,NaN,NaN,0.00,0.0,06/30/2020,0.00,0.0,50126.74,40699.28


In [7]:
def map_candidate_to_seat_type(df):
    
    ici_map = {'C': 'Challenger', 'I': 'Incumbent', 'O': 'Open Seat'}
    df['CAND_ICI_FULL'] = df['CAND_ICI'].map(ici_map)
    
    return df

all_candidates = map_candidate_to_seat_type(all_candidates)
all_candidates.head()

,CAND_ID,CAND_NAME,CAND_ICI,PTY_CD,CAND_PTY_AFFILIATION,TTL_RECEIPTS,TRANS_FROM_AUTH,TTL_DISB,TRANS_TO_AUTH,COH_BOP,COH_COP,CAND_CONTRIB,CAND_LOANS,OTHER_LOANS,CAND_LOAN_REPAY,OTHER_LOAN_REPAY,DEBTS_OWED_BY,TTL_INDIV_CONTRIB,CAND_OFFICE_ST,CAND_OFFICE_DISTRICT,SPEC_ELECTION,PRIM_ELECTION,RUN_ELECTION,GEN_ELECTION,GEN_ELECTION_PRECENT,OTHER_POL_CMTE_CONTRIB,POL_PTY_CONTRIB,CVG_END_DT,INDIV_REFUNDS,CMTE_REFUNDS,TTL_RECEIPTS_CORRECTED,TTL_DISB_CORRECTED,CAND_ICI_FULL
0,H8AK00132,"SHEIN, DIMITRI",C,1,DEM,0.00,0.00,367.52,0.0,367.52,0.00,0.00,0.0,0.0,367.52,0.0,0.00,0.00,AK,0,NaN,NaN,NaN,NaN,NaN,0.00,0.0,09/30/2019,0.00,0.0,0.00,367.52,Challenger
1,H6AK00045,"YOUNG, DONALD E",I,2,REP,1012401.48,130245.48,325545.51,0.0,116720.12,803576.09,0.00,0.0,0.0,0.00,0.0,0.00,417190.55,AK,0,NaN,NaN,NaN,NaN,NaN,458194.63,0.0,03/31/2020,0.00,2000.0,1012401.48,325545.51,Incumbent
2,H8AK01031,"NELSON, THOMAS JOHN",C,2,REP,0.00,0.00,466.51,0.0,466.51,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00,AK,0,NaN,NaN,NaN,NaN,NaN,0.00,0.0,03/31/2019,0.00,0.0,0.00,466.51,Challenger
3,H8AK00140,"GALVIN, ALYSE",C,3,IND,1358372.81,0.00,403275.92,0.0,6245.09,961341.98,2955.81,0.0,0.0,0.00,0.0,56940.58,1279739.80,AK,0,NaN,NaN,NaN,NaN,NaN,38350.00,0.0,03/31/2020,7836.19,0.0,1358372.81,403275.92,Challenger
4,H0AL01097,"AVERHART, JAMES",O,1,DEM,50126.74,0.00,40699.28,0.0,0.00,9427.46,0.00,26845.0,0.0,0.00,0.0,26845.00,23281.74,AL,1,NaN,NaN,NaN,NaN,NaN,0.00,0.0,06/30/2020,0.00,0.0,50126.74,40699.28,Open Seat


In [8]:
party_codes = read_table("https://www.fec.gov/campaign-finance-data/party-code-descriptions/")

def map_candidate_to_party(df):
    
    df['CAND_PTY_AFFILIATION_FULL'] = df['CAND_PTY_AFFILIATION'].map(party_codes)
    
    return df

all_candidates = map_candidate_to_party(all_candidates)
all_candidates.head()

,CAND_ID,CAND_NAME,CAND_ICI,PTY_CD,CAND_PTY_AFFILIATION,TTL_RECEIPTS,TRANS_FROM_AUTH,TTL_DISB,TRANS_TO_AUTH,COH_BOP,COH_COP,CAND_CONTRIB,CAND_LOANS,OTHER_LOANS,CAND_LOAN_REPAY,OTHER_LOAN_REPAY,DEBTS_OWED_BY,TTL_INDIV_CONTRIB,CAND_OFFICE_ST,CAND_OFFICE_DISTRICT,SPEC_ELECTION,PRIM_ELECTION,RUN_ELECTION,GEN_ELECTION,GEN_ELECTION_PRECENT,OTHER_POL_CMTE_CONTRIB,POL_PTY_CONTRIB,CVG_END_DT,INDIV_REFUNDS,CMTE_REFUNDS,TTL_RECEIPTS_CORRECTED,TTL_DISB_CORRECTED,CAND_ICI_FULL,CAND_PTY_AFFILIATION_FULL
0,H8AK00132,"SHEIN, DIMITRI",C,1,DEM,0.00,0.00,367.52,0.0,367.52,0.00,0.00,0.0,0.0,367.52,0.0,0.00,0.00,AK,0,NaN,NaN,NaN,NaN,NaN,0.00,0.0,09/30/2019,0.00,0.0,0.00,367.52,Challenger,Democratic Party
1,H6AK00045,"YOUNG, DONALD E",I,2,REP,1012401.48,130245.48,325545.51,0.0,116720.12,803576.09,0.00,0.0,0.0,0.00,0.0,0.00,417190.55,AK,0,NaN,NaN,NaN,NaN,NaN,458194.63,0.0,03/31/2020,0.00,2000.0,1012401.48,325545.51,Incumbent,Republican Party
2,H8AK01031,"NELSON, THOMAS JOHN",C,2,REP,0.00,0.00,466.51,0.0,466.51,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00,AK,0,NaN,NaN,NaN,NaN,NaN,0.00,0.0,03/31/2019,0.00,0.0,0.00,466.51,Challenger,Republican Party
3,H8AK00140,"GALVIN, ALYSE",C,3,IND,1358372.81,0.00,403275.92,0.0,6245.09,961341.98,2955.81,0.0,0.0,0.00,0.0,56940.58,1279739.80,AK,0,NaN,NaN,NaN,NaN,NaN,38350.00,0.0,03/31/2020,7836.19,0.0,1358372.81,403275.92,Challenger,Independent
4,H0AL01097,"AVERHART, JAMES",O,1,DEM,50126.74,0.00,40699.28,0.0,0.00,9427.46,0.00,26845.0,0.0,0.00,0.0,26845.00,23281.74,AL,1,NaN,NaN,NaN,NaN,NaN,0.00,0.0,06/30/2020,0.00,0.0,50126.74,40699.28,Open Seat,Democratic Party


In [9]:
def correct_candidate_district_and_office(df):
    
    if df['CAND_OFFICE_DISTRICT'].dtype == 'int64':
        df['CAND_OFFICE_DISTRICT'] = df['CAND_OFFICE_DISTRICT'].map(lambda x: str(x).zfill(2) if not pd.isna(x) else np.nan)
    elif df['CAND_OFFICE_DISTRICT'].dtype == 'float':
        df['CAND_OFFICE_DISTRICT'] = df['CAND_OFFICE_DISTRICT'].map(lambda x: str(int(x)).zfill(2) if not pd.isna(x) else np.nan)
    return df

all_candidates = correct_candidate_district_and_office(all_candidates)
all_candidates.head()

,CAND_ID,CAND_NAME,CAND_ICI,PTY_CD,CAND_PTY_AFFILIATION,TTL_RECEIPTS,TRANS_FROM_AUTH,TTL_DISB,TRANS_TO_AUTH,COH_BOP,COH_COP,CAND_CONTRIB,CAND_LOANS,OTHER_LOANS,CAND_LOAN_REPAY,OTHER_LOAN_REPAY,DEBTS_OWED_BY,TTL_INDIV_CONTRIB,CAND_OFFICE_ST,CAND_OFFICE_DISTRICT,SPEC_ELECTION,PRIM_ELECTION,RUN_ELECTION,GEN_ELECTION,GEN_ELECTION_PRECENT,OTHER_POL_CMTE_CONTRIB,POL_PTY_CONTRIB,CVG_END_DT,INDIV_REFUNDS,CMTE_REFUNDS,TTL_RECEIPTS_CORRECTED,TTL_DISB_CORRECTED,CAND_ICI_FULL,CAND_PTY_AFFILIATION_FULL
0,H8AK00132,"SHEIN, DIMITRI",C,1,DEM,0.00,0.00,367.52,0.0,367.52,0.00,0.00,0.0,0.0,367.52,0.0,0.00,0.00,AK,00,NaN,NaN,NaN,NaN,NaN,0.00,0.0,09/30/2019,0.00,0.0,0.00,367.52,Challenger,Democratic Party
1,H6AK00045,"YOUNG, DONALD E",I,2,REP,1012401.48,130245.48,325545.51,0.0,116720.12,803576.09,0.00,0.0,0.0,0.00,0.0,0.00,417190.55,AK,00,NaN,NaN,NaN,NaN,NaN,458194.63,0.0,03/31/2020,0.00,2000.0,1012401.48,325545.51,Incumbent,Republican Party
2,H8AK01031,"NELSON, THOMAS JOHN",C,2,REP,0.00,0.00,466.51,0.0,466.51,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00,AK,00,NaN,NaN,NaN,NaN,NaN,0.00,0.0,03/31/2019,0.00,0.0,0.00,466.51,Challenger,Republican Party
3,H8AK00140,"GALVIN, ALYSE",C,3,IND,1358372.81,0.00,403275.92,0.0,6245.09,961341.98,2955.81,0.0,0.0,0.00,0.0,56940.58,1279739.80,AK,00,NaN,NaN,NaN,NaN,NaN,38350.00,0.0,03/31/2020,7836.19,0.0,1358372.81,403275.92,Challenger,Independent
4,H0AL01097,"AVERHART, JAMES",O,1,DEM,50126.74,0.00,40699.28,0.0,0.00,9427.46,0.00,26845.0,0.0,0.00,0.0,26845.00,23281.74,AL,01,NaN,NaN,NaN,NaN,NaN,0.00,0.0,06/30/2020,0.00,0.0,50126.74,40699.28,Open Seat,Democratic Party


In [10]:
candidates = all_candidates[(all_candidates['CAND_NAME'].str.contains(f"{candidate1}", case=False)) | 
               (all_candidates['CAND_NAME'].str.contains(f"{candidate2}", case=False))]
candidates

,CAND_ID,CAND_NAME,CAND_ICI,PTY_CD,CAND_PTY_AFFILIATION,TTL_RECEIPTS,TRANS_FROM_AUTH,TTL_DISB,TRANS_TO_AUTH,COH_BOP,COH_COP,CAND_CONTRIB,CAND_LOANS,OTHER_LOANS,CAND_LOAN_REPAY,OTHER_LOAN_REPAY,DEBTS_OWED_BY,TTL_INDIV_CONTRIB,CAND_OFFICE_ST,CAND_OFFICE_DISTRICT,SPEC_ELECTION,PRIM_ELECTION,RUN_ELECTION,GEN_ELECTION,GEN_ELECTION_PRECENT,OTHER_POL_CMTE_CONTRIB,POL_PTY_CONTRIB,CVG_END_DT,INDIV_REFUNDS,CMTE_REFUNDS,TTL_RECEIPTS_CORRECTED,TTL_DISB_CORRECTED,CAND_ICI_FULL,CAND_PTY_AFFILIATION_FULL
3512,S0NC00202,"CUNNINGHAM, CAL",C,1,DEM,7716897.31,756819.84,4716418.25,0.00,0.00,3000479.06,0.0,200000.0,0.0,200000.0,0.0,0.0,6268724.11,NC,00,NaN,NaN,NaN,NaN,NaN,438325.00,49600.0,03/31/2020,29751.69,0.00,7716897.31,4716418.25,Challenger,Democratic Party
3523,S4NC00162,"TILLIS, THOM R. SEN.",I,2,REP,8269271.66,1152018.96,3811302.27,6441.91,2025444.43,6483413.82,0.0,0.0,0.0,0.0,0.0,0.0,4798506.58,NC,00,NaN,NaN,NaN,NaN,NaN,2278244.39,0.0,06/09/2020,102493.33,14425.67,7117252.70,3804860.36,Incumbent,Republican Party


In [11]:
from IPython.display import HTML

candidates_summary = candidates.groupby('CAND_NAME').agg({'TTL_RECEIPTS_CORRECTED': 'sum',
                                                         'TTL_DISB_CORRECTED': 'sum', 'COH_COP': 'sum',
                                                         'CVG_END_DT': 'max'}).reset_index()

candidates_summary['Raised'] = '$' + candidates_summary['TTL_RECEIPTS_CORRECTED'].map(render_human_format).astype('str')
candidates_summary['Spent'] = '$' + candidates_summary['TTL_DISB_CORRECTED'].map(render_human_format).astype('str')

candidates_summary['Cash on Hand'] = '$' + candidates_summary['COH_COP'].map(render_human_format).astype('str')
candidates_summary['Last Report date'] = pd.to_datetime(candidates_summary['CVG_END_DT']).dt.strftime('%b %d %Y')

HTML(candidates_summary.rename(columns={'CAND_NAME': 'Candidate'})[['Candidate', 'Raised', 'Spent', 'Cash on Hand', 'Last Report date']].to_html(index=False))

Candidate,Raised,Spent,Cash on Hand,Last Report date
"CUNNINGHAM, CAL",$7.72M,$4.72M,$3.00M,Mar 31 2020
"TILLIS, THOM R. SEN.",$7.12M,$3.80M,$6.48M,Jun 09 2020


In [12]:
republican = candidates[candidates['CAND_PTY_AFFILIATION'] == 'REP']['CAND_NAME'].values[0]
democrat = candidates[candidates['CAND_PTY_AFFILIATION'] == 'DEM']['CAND_NAME'].values[0]
print(republican)
democrat

TILLIS, THOM R. SEN.


'CUNNINGHAM, CAL'

In [13]:
candidates['color'] = candidates['CAND_PTY_AFFILIATION'].map({'DEM': 'blue', 'REP': 'red'})
candidates['second_color'] = candidates['CAND_PTY_AFFILIATION'].map({'DEM': 'lightblue', 'REP': 'pink'})
color_map = dict()
secondary_color_map = dict()
for row in candidates[['CAND_NAME', 'color', 'second_color']].values:
    color_map[row[0]] = row[1]
    secondary_color_map[f"For {row[0]}"] = row[1]
    secondary_color_map[f"Against {row[0]}"] = row[2]
print(color_map)
#candidates.head()
secondary_color_map

{'CUNNINGHAM, CAL': 'blue', 'TILLIS, THOM R. SEN.': 'red'}


{'For CUNNINGHAM, CAL': 'blue',
 'Against CUNNINGHAM, CAL': 'lightblue',
 'For TILLIS, THOM R. SEN.': 'red',
 'Against TILLIS, THOM R. SEN.': 'pink'}

In [14]:
candidates_desc = read_table("https://www.fec.gov/campaign-finance-data/all-candidates-file-description/")
candidates_desc = {key.strip():val.strip() for key, val in candidates_desc.items()}
candidates_desc

{'Column name': 'Field name',
 'CAND_ID': 'Candidate identification',
 'CAND_NAME': 'Candidate name',
 'CAND_ICI': 'Incumbent challenger status',
 'PTY_CD': 'Party code',
 'CAND_PTY_AFFILIATION': 'Party affiliation',
 'TTL_RECEIPTS': 'Total receipts',
 'TRANS_FROM_AUTH': 'Transfers from authorized committees',
 'TTL_DISB': 'Total disbursements',
 'TRANS_TO_AUTH': 'Transfers to authorized committees',
 'COH_BOP': 'Beginning cash',
 'COH_COP': 'Ending cash',
 'CAND_CONTRIB': 'Contributions from candidate',
 'CAND_LOANS': 'Loans from candidate',
 'OTHER_LOANS': 'Other loans',
 'CAND_LOAN_REPAY': 'Candidate loan repayments',
 'OTHER_LOAN_REPAY': 'Other loan repayments',
 'DEBTS_OWED_BY': 'Debts owed by',
 'TTL_INDIV_CONTRIB': 'Total individual contributions',
 'CAND_OFFICE_ST': 'Candidate state',
 'CAND_OFFICE_DISTRICT': 'Candidate district',
 'SPEC_ELECTION': 'Special election status',
 'PRIM_ELECTION': 'Primary election status',
 'RUN_ELECTION': 'Runoff election status',
 'GEN_ELECTION':

In [15]:
candidates_desc.update({'TTL_RECEIPTS_CORRECTED': 'Total Receipts', 'TTL_DISB_CORRECTED': 'Total Disbursements'})
camp_finance_summary_cols = ['TTL_RECEIPTS_CORRECTED', 'TTL_DISB_CORRECTED', 'TRANS_FROM_AUTH', 'COH_BOP', 'COH_COP', 'CAND_CONTRIB', 'CAND_LOANS', 'OTHER_LOANS', 'DEBTS_OWED_BY', 'TTL_INDIV_CONTRIB', 'POL_PTY_CONTRIB', 'OTHER_POL_CMTE_CONTRIB']

focused_candidates = candidates.groupby('CAND_NAME').agg({x: 'sum' for x in camp_finance_summary_cols}).reset_index()
total_finance = focused_candidates.set_index('CAND_NAME')[camp_finance_summary_cols].transpose()
total_finance = total_finance.rename(candidates_desc)
total_finance.iplot(kind='bar', title='Campaign finance summary', colors=color_map)

## Contributions & expenditures from committees

In [16]:
contributions_from_committees = pd.read_csv(f"../data/20192020-FEC/Contributions from committees to candidates  independent expenditures.csv")
contributions_from_committees.head()

/Users/sahluwalia/acre/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3257: DtypeWarning:

Columns (11,12) have mixed types.Specify dtype option on import or set low_memory=False.



,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,ZIP_CODE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,CAND_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID
0,C00567180,T,TER,P2020,201901099143774199,24K,PAC,TED YOHO FOR CONGRESS,GAINESVILLE,FL,32608.0,NaN,NaN,1082019.0,1880,C00494583,H2FL06109,SB23.4447,1303604,NaN,NaN,4021920191640423011
1,C00104885,A,TER,G2020,201901289144031511,24K,CCM,TEAM GRAHAM INC,COLUMBIA,SC,29202.0,NaN,NaN,12202018.0,3000,C00458828,H4SC03087,SB23.17757,1307636,NaN,NaN,4022220191643444985
2,C00104885,A,TER,P2022,201901289144031512,24K,CCM,TIM SCOTT FOR SENATE,CHARLESTON,SC,29407.0,NaN,NaN,12202018.0,1000,C00540302,H0SC01279,SB23.17756,1307636,NaN,NaN,4022220191643444987
3,C00104885,A,TER,P2020,201901289144031511,24K,CCM,FRIENDS OF JIM CLYBURN,COLUMBIA,SC,29211.0,NaN,NaN,1072019.0,470,C00255562,H2SC02042,SB23.17755,1307636,NaN,NaN,4022220191643444981
4,C00688408,T,TER,G2018,201901319144305867,24E,ORG,JACKSON ADVOCATE,JACKSON,MS,39207.0,NaN,NaN,12122018.0,1000,S8MS00287,S8MS00287,SE.4233,1310906,NaN,NaN,4022420191643632157


In [17]:
def fix_zip_codes(col, df):
    if df[col].dtype == 'float':
        df[col] = df[col].map(lambda x: str(int(x)).zfill(5) if not pd.isna(x) else np.nan)
    elif df[col].dtype == 'int64':
        df[col] = df[col].map(lambda x: str(x).zfill(5) if not pd.isna(x) else np.nan)
    return df
contributions_from_committees = fix_zip_codes('ZIP_CODE', contributions_from_committees)
contributions_from_committees.head()

,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,ZIP_CODE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,CAND_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID
0,C00567180,T,TER,P2020,201901099143774199,24K,PAC,TED YOHO FOR CONGRESS,GAINESVILLE,FL,32608,NaN,NaN,1082019.0,1880,C00494583,H2FL06109,SB23.4447,1303604,NaN,NaN,4021920191640423011
1,C00104885,A,TER,G2020,201901289144031511,24K,CCM,TEAM GRAHAM INC,COLUMBIA,SC,29202,NaN,NaN,12202018.0,3000,C00458828,H4SC03087,SB23.17757,1307636,NaN,NaN,4022220191643444985
2,C00104885,A,TER,P2022,201901289144031512,24K,CCM,TIM SCOTT FOR SENATE,CHARLESTON,SC,29407,NaN,NaN,12202018.0,1000,C00540302,H0SC01279,SB23.17756,1307636,NaN,NaN,4022220191643444987
3,C00104885,A,TER,P2020,201901289144031511,24K,CCM,FRIENDS OF JIM CLYBURN,COLUMBIA,SC,29211,NaN,NaN,1072019.0,470,C00255562,H2SC02042,SB23.17755,1307636,NaN,NaN,4022220191643444981
4,C00688408,T,TER,G2018,201901319144305867,24E,ORG,JACKSON ADVOCATE,JACKSON,MS,39207,NaN,NaN,12122018.0,1000,S8MS00287,S8MS00287,SE.4233,1310906,NaN,NaN,4022420191643632157


In [18]:
def map_amndt_ind(df):
    df['AMNDT_IND_FULL'] = df['AMNDT_IND'].map({'N': 'New', 'A': 'Amendment', 'T': 'Termination'})
    return df
contributions_from_committees = map_amndt_ind(contributions_from_committees)
contributions_from_committees.head()

,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,ZIP_CODE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,CAND_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID,AMNDT_IND_FULL
0,C00567180,T,TER,P2020,201901099143774199,24K,PAC,TED YOHO FOR CONGRESS,GAINESVILLE,FL,32608,NaN,NaN,1082019.0,1880,C00494583,H2FL06109,SB23.4447,1303604,NaN,NaN,4021920191640423011,Termination
1,C00104885,A,TER,G2020,201901289144031511,24K,CCM,TEAM GRAHAM INC,COLUMBIA,SC,29202,NaN,NaN,12202018.0,3000,C00458828,H4SC03087,SB23.17757,1307636,NaN,NaN,4022220191643444985,Amendment
2,C00104885,A,TER,P2022,201901289144031512,24K,CCM,TIM SCOTT FOR SENATE,CHARLESTON,SC,29407,NaN,NaN,12202018.0,1000,C00540302,H0SC01279,SB23.17756,1307636,NaN,NaN,4022220191643444987,Amendment
3,C00104885,A,TER,P2020,201901289144031511,24K,CCM,FRIENDS OF JIM CLYBURN,COLUMBIA,SC,29211,NaN,NaN,1072019.0,470,C00255562,H2SC02042,SB23.17755,1307636,NaN,NaN,4022220191643444981,Amendment
4,C00688408,T,TER,G2018,201901319144305867,24E,ORG,JACKSON ADVOCATE,JACKSON,MS,39207,NaN,NaN,12122018.0,1000,S8MS00287,S8MS00287,SE.4233,1310906,NaN,NaN,4022420191643632157,Termination


In [19]:
contributions_from_committees = contributions_from_committees[contributions_from_committees.AMNDT_IND == "N"]

In [20]:
report_type_map = read_table("https://www.fec.gov/campaign-finance-data/report-type-code-descriptions/")
def map_report_type(df):
    df['RPT_TP_FULL'] = df['RPT_TP'].map(report_type_map)
    return df
contributions_from_committees = map_report_type(contributions_from_committees)
contributions_from_committees.head()

,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,ZIP_CODE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,CAND_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID,AMNDT_IND_FULL,RPT_TP_FULL
5,C00325324,N,M2,P2020,201902049145458880,24K,CCM,EMMER FOR CONGRESS,ANOKA,MN,55303,NaN,NaN,1252019.0,1500,C00545749,H4MN06087,B27CCDF382AE3419EB12,1313984,NaN,NaN,4022620191643762236,New,February monthly
6,C00414425,N,M2,P,201902019145450791,24K,CCM,FRIENDS OF MARK WARNER,ALEXANDRIA,VA,22314,NaN,NaN,1292019.0,1000,C00438713,S6VA00093,SB23.9011,1313349,NaN,NaN,4022620191643762178,New,February monthly
7,C00366013,N,M2,P2020,201902049145460163,24K,CCM,CLAY JR. FOR CONGRESS,SAINT LOUIS,MO,631080544,NaN,NaN,1102019.0,1000,C00346080,H0MO01066,BCC5E5592BA664439810,1314222,NaN,NaN,4022620191643761316,New,February monthly
8,C00366013,N,M2,P2020,201902049145460162,24K,CCM,MALONEY FOR CONGRESS,NEW YORK,NY,101281326,NaN,NaN,1022019.0,2500,C00273169,H2NY14037,BE5946F36B95C4B1AA76,1314222,NaN,NaN,4022620191643761310,New,February monthly
9,C00366013,N,M2,P2020,201902049145460162,24K,CCM,MCHENRY FOR CONGRESS,GASTONIA,NC,280532165,NaN,NaN,1102019.0,1000,C00393629,H4NC10047,B4A12FABC952A45BFAA6,1314222,NaN,NaN,4022620191643761312,New,February monthly


In [21]:
election_type_map = {
    'P': 'Primary',
    'G': 'General',
    'O': 'Other',
    'C': 'Convention',
    'R': 'Runoff',
    'S': 'Special',
    'E': 'Recount'
}
def parse_transaction_pgi(df):
    df['ELECTION_TYPE'] = df['TRANSACTION_PGI'].astype('object').str[0].map(election_type_map)
    df['ELECTION_YEAR'] = df['TRANSACTION_PGI'].astype('object').str[1:].map(
        lambda x: int(x) if x and not pd.isnull(x) else np.nan).fillna(0).astype('int')
    return df
contributions_from_committees = parse_transaction_pgi(contributions_from_committees)
contributions_from_committees.head()

,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,ZIP_CODE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,CAND_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID,AMNDT_IND_FULL,RPT_TP_FULL,ELECTION_TYPE,ELECTION_YEAR
5,C00325324,N,M2,P2020,201902049145458880,24K,CCM,EMMER FOR CONGRESS,ANOKA,MN,55303,NaN,NaN,1252019.0,1500,C00545749,H4MN06087,B27CCDF382AE3419EB12,1313984,NaN,NaN,4022620191643762236,New,February monthly,Primary,2020
6,C00414425,N,M2,P,201902019145450791,24K,CCM,FRIENDS OF MARK WARNER,ALEXANDRIA,VA,22314,NaN,NaN,1292019.0,1000,C00438713,S6VA00093,SB23.9011,1313349,NaN,NaN,4022620191643762178,New,February monthly,Primary,0
7,C00366013,N,M2,P2020,201902049145460163,24K,CCM,CLAY JR. FOR CONGRESS,SAINT LOUIS,MO,631080544,NaN,NaN,1102019.0,1000,C00346080,H0MO01066,BCC5E5592BA664439810,1314222,NaN,NaN,4022620191643761316,New,February monthly,Primary,2020
8,C00366013,N,M2,P2020,201902049145460162,24K,CCM,MALONEY FOR CONGRESS,NEW YORK,NY,101281326,NaN,NaN,1022019.0,2500,C00273169,H2NY14037,BE5946F36B95C4B1AA76,1314222,NaN,NaN,4022620191643761310,New,February monthly,Primary,2020
9,C00366013,N,M2,P2020,201902049145460162,24K,CCM,MCHENRY FOR CONGRESS,GASTONIA,NC,280532165,NaN,NaN,1102019.0,1000,C00393629,H4NC10047,B4A12FABC952A45BFAA6,1314222,NaN,NaN,4022620191643761312,New,February monthly,Primary,2020


In [22]:
transaction_type_map = read_table("https://www.fec.gov/campaign-finance-data/transaction-type-code-descriptions/")

def map_transaction_type(df):
    
    df['TRANSACTION_TP_FULL'] = df['TRANSACTION_TP'].map(transaction_type_map)

    return df

contributions_from_committees = map_transaction_type(contributions_from_committees)
contributions_from_committees.head()

,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,ZIP_CODE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,CAND_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID,AMNDT_IND_FULL,RPT_TP_FULL,ELECTION_TYPE,ELECTION_YEAR,TRANSACTION_TP_FULL
5,C00325324,N,M2,P2020,201902049145458880,24K,CCM,EMMER FOR CONGRESS,ANOKA,MN,55303,NaN,NaN,1252019.0,1500,C00545749,H4MN06087,B27CCDF382AE3419EB12,1313984,NaN,NaN,4022620191643762236,New,February monthly,Primary,2020,Contribution made to nonaffiliated committee
6,C00414425,N,M2,P,201902019145450791,24K,CCM,FRIENDS OF MARK WARNER,ALEXANDRIA,VA,22314,NaN,NaN,1292019.0,1000,C00438713,S6VA00093,SB23.9011,1313349,NaN,NaN,4022620191643762178,New,February monthly,Primary,0,Contribution made to nonaffiliated committee
7,C00366013,N,M2,P2020,201902049145460163,24K,CCM,CLAY JR. FOR CONGRESS,SAINT LOUIS,MO,631080544,NaN,NaN,1102019.0,1000,C00346080,H0MO01066,BCC5E5592BA664439810,1314222,NaN,NaN,4022620191643761316,New,February monthly,Primary,2020,Contribution made to nonaffiliated committee
8,C00366013,N,M2,P2020,201902049145460162,24K,CCM,MALONEY FOR CONGRESS,NEW YORK,NY,101281326,NaN,NaN,1022019.0,2500,C00273169,H2NY14037,BE5946F36B95C4B1AA76,1314222,NaN,NaN,4022620191643761310,New,February monthly,Primary,2020,Contribution made to nonaffiliated committee
9,C00366013,N,M2,P2020,201902049145460162,24K,CCM,MCHENRY FOR CONGRESS,GASTONIA,NC,280532165,NaN,NaN,1102019.0,1000,C00393629,H4NC10047,B4A12FABC952A45BFAA6,1314222,NaN,NaN,4022620191643761312,New,February monthly,Primary,2020,Contribution made to nonaffiliated committee


In [23]:
entity_type_map = {
    'CAN': 'Candidate',
    'CCM': 'Candidate Committee',
    'COM': 'Committee',
    'IND': 'Individual (a person)',
    'ORG': 'Organization (not a committee and not a person)',
    'PAC': 'Political Action Committee',
    'PTY': 'Party Organization'
}
def map_entity_type(df):
    df['ENTITY_TP_FULL'] = df['ENTITY_TP'].map(entity_type_map)
    return df
contributions_from_committees = map_entity_type(contributions_from_committees)
contributions_from_committees.head()

,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,ZIP_CODE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,CAND_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID,AMNDT_IND_FULL,RPT_TP_FULL,ELECTION_TYPE,ELECTION_YEAR,TRANSACTION_TP_FULL,ENTITY_TP_FULL
5,C00325324,N,M2,P2020,201902049145458880,24K,CCM,EMMER FOR CONGRESS,ANOKA,MN,55303,NaN,NaN,1252019.0,1500,C00545749,H4MN06087,B27CCDF382AE3419EB12,1313984,NaN,NaN,4022620191643762236,New,February monthly,Primary,2020,Contribution made to nonaffiliated committee,Candidate Committee
6,C00414425,N,M2,P,201902019145450791,24K,CCM,FRIENDS OF MARK WARNER,ALEXANDRIA,VA,22314,NaN,NaN,1292019.0,1000,C00438713,S6VA00093,SB23.9011,1313349,NaN,NaN,4022620191643762178,New,February monthly,Primary,0,Contribution made to nonaffiliated committee,Candidate Committee
7,C00366013,N,M2,P2020,201902049145460163,24K,CCM,CLAY JR. FOR CONGRESS,SAINT LOUIS,MO,631080544,NaN,NaN,1102019.0,1000,C00346080,H0MO01066,BCC5E5592BA664439810,1314222,NaN,NaN,4022620191643761316,New,February monthly,Primary,2020,Contribution made to nonaffiliated committee,Candidate Committee
8,C00366013,N,M2,P2020,201902049145460162,24K,CCM,MALONEY FOR CONGRESS,NEW YORK,NY,101281326,NaN,NaN,1022019.0,2500,C00273169,H2NY14037,BE5946F36B95C4B1AA76,1314222,NaN,NaN,4022620191643761310,New,February monthly,Primary,2020,Contribution made to nonaffiliated committee,Candidate Committee
9,C00366013,N,M2,P2020,201902049145460162,24K,CCM,MCHENRY FOR CONGRESS,GASTONIA,NC,280532165,NaN,NaN,1102019.0,1000,C00393629,H4NC10047,B4A12FABC952A45BFAA6,1314222,NaN,NaN,4022620191643761312,New,February monthly,Primary,2020,Contribution made to nonaffiliated committee,Candidate Committee


In [24]:
# We remove the original rows that have been amended later on
contributions_from_committees[['TRAN_ID_parent', 'TRAN_ID_child']] = contributions_from_committees['TRAN_ID'].str.split('.', n=1, expand=True)

merged = contributions_from_committees.merge(contributions_from_committees, on=['CMTE_ID', 'CAND_ID', 'TRAN_ID_parent'])

amended_contribs = merged[(merged['AMNDT_IND_x'] == 'N') & (merged['AMNDT_IND_y'] == 'A') & (merged['TRANSACTION_DT_y'] > merged['TRANSACTION_DT_x'])]['SUB_ID_x']

# sanity check
contributions_from_committees = contributions_from_committees[~contributions_from_committees['SUB_ID'].isin(amended_contribs)]

contributions_from_committees.head()

,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,ZIP_CODE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,CAND_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID,AMNDT_IND_FULL,RPT_TP_FULL,ELECTION_TYPE,ELECTION_YEAR,TRANSACTION_TP_FULL,ENTITY_TP_FULL,TRAN_ID_parent,TRAN_ID_child
5,C00325324,N,M2,P2020,201902049145458880,24K,CCM,EMMER FOR CONGRESS,ANOKA,MN,55303,NaN,NaN,1252019.0,1500,C00545749,H4MN06087,B27CCDF382AE3419EB12,1313984,NaN,NaN,4022620191643762236,New,February monthly,Primary,2020,Contribution made to nonaffiliated committee,Candidate Committee,B27CCDF382AE3419EB12,None
6,C00414425,N,M2,P,201902019145450791,24K,CCM,FRIENDS OF MARK WARNER,ALEXANDRIA,VA,22314,NaN,NaN,1292019.0,1000,C00438713,S6VA00093,SB23.9011,1313349,NaN,NaN,4022620191643762178,New,February monthly,Primary,0,Contribution made to nonaffiliated committee,Candidate Committee,SB23,9011
7,C00366013,N,M2,P2020,201902049145460163,24K,CCM,CLAY JR. FOR CONGRESS,SAINT LOUIS,MO,631080544,NaN,NaN,1102019.0,1000,C00346080,H0MO01066,BCC5E5592BA664439810,1314222,NaN,NaN,4022620191643761316,New,February monthly,Primary,2020,Contribution made to nonaffiliated committee,Candidate Committee,BCC5E5592BA664439810,None
8,C00366013,N,M2,P2020,201902049145460162,24K,CCM,MALONEY FOR CONGRESS,NEW YORK,NY,101281326,NaN,NaN,1022019.0,2500,C00273169,H2NY14037,BE5946F36B95C4B1AA76,1314222,NaN,NaN,4022620191643761310,New,February monthly,Primary,2020,Contribution made to nonaffiliated committee,Candidate Committee,BE5946F36B95C4B1AA76,None
9,C00366013,N,M2,P2020,201902049145460162,24K,CCM,MCHENRY FOR CONGRESS,GASTONIA,NC,280532165,NaN,NaN,1102019.0,1000,C00393629,H4NC10047,B4A12FABC952A45BFAA6,1314222,NaN,NaN,4022620191643761312,New,February monthly,Primary,2020,Contribution made to nonaffiliated committee,Candidate Committee,B4A12FABC952A45BFAA6,None


In [25]:
contributions_from_committees['TRANSACTION_DT'] = pd.to_datetime(
    contributions_from_committees['TRANSACTION_DT'].fillna(0).astype('int').astype('str')
    .replace('0', np.NaN).str.zfill(8), format='%m%d%Y')
contributions_from_committees.head()

,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,ZIP_CODE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,CAND_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID,AMNDT_IND_FULL,RPT_TP_FULL,ELECTION_TYPE,ELECTION_YEAR,TRANSACTION_TP_FULL,ENTITY_TP_FULL,TRAN_ID_parent,TRAN_ID_child
5,C00325324,N,M2,P2020,201902049145458880,24K,CCM,EMMER FOR CONGRESS,ANOKA,MN,55303,NaN,NaN,2019-01-25,1500,C00545749,H4MN06087,B27CCDF382AE3419EB12,1313984,NaN,NaN,4022620191643762236,New,February monthly,Primary,2020,Contribution made to nonaffiliated committee,Candidate Committee,B27CCDF382AE3419EB12,None
6,C00414425,N,M2,P,201902019145450791,24K,CCM,FRIENDS OF MARK WARNER,ALEXANDRIA,VA,22314,NaN,NaN,2019-01-29,1000,C00438713,S6VA00093,SB23.9011,1313349,NaN,NaN,4022620191643762178,New,February monthly,Primary,0,Contribution made to nonaffiliated committee,Candidate Committee,SB23,9011
7,C00366013,N,M2,P2020,201902049145460163,24K,CCM,CLAY JR. FOR CONGRESS,SAINT LOUIS,MO,631080544,NaN,NaN,2019-01-10,1000,C00346080,H0MO01066,BCC5E5592BA664439810,1314222,NaN,NaN,4022620191643761316,New,February monthly,Primary,2020,Contribution made to nonaffiliated committee,Candidate Committee,BCC5E5592BA664439810,None
8,C00366013,N,M2,P2020,201902049145460162,24K,CCM,MALONEY FOR CONGRESS,NEW YORK,NY,101281326,NaN,NaN,2019-01-02,2500,C00273169,H2NY14037,BE5946F36B95C4B1AA76,1314222,NaN,NaN,4022620191643761310,New,February monthly,Primary,2020,Contribution made to nonaffiliated committee,Candidate Committee,BE5946F36B95C4B1AA76,None
9,C00366013,N,M2,P2020,201902049145460162,24K,CCM,MCHENRY FOR CONGRESS,GASTONIA,NC,280532165,NaN,NaN,2019-01-10,1000,C00393629,H4NC10047,B4A12FABC952A45BFAA6,1314222,NaN,NaN,4022620191643761312,New,February monthly,Primary,2020,Contribution made to nonaffiliated committee,Candidate Committee,B4A12FABC952A45BFAA6,None


In [26]:
committee_contribs = contributions_from_committees.merge(candidates, on='CAND_ID')
committee_contribs.head()

,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,ZIP_CODE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,CAND_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID,AMNDT_IND_FULL,RPT_TP_FULL,ELECTION_TYPE,ELECTION_YEAR,TRANSACTION_TP_FULL,ENTITY_TP_FULL,TRAN_ID_parent,TRAN_ID_child,CAND_NAME,CAND_ICI,PTY_CD,CAND_PTY_AFFILIATION,TTL_RECEIPTS,TRANS_FROM_AUTH,TTL_DISB,TRANS_TO_AUTH,COH_BOP,COH_COP,CAND_CONTRIB,CAND_LOANS,OTHER_LOANS,CAND_LOAN_REPAY,OTHER_LOAN_REPAY,DEBTS_OWED_BY,TTL_INDIV_CONTRIB,CAND_OFFICE_ST,CAND_OFFICE_DISTRICT,SPEC_ELECTION,PRIM_ELECTION,RUN_ELECTION,GEN_ELECTION,GEN_ELECTION_PRECENT,OTHER_POL_CMTE_CONTRIB,POL_PTY_CONTRIB,CVG_END_DT,INDIV_REFUNDS,CMTE_REFUNDS,TTL_RECEIPTS_CORRECTED,TTL_DISB_CORRECTED,CAND_ICI_FULL,CAND_PTY_AFFILIATION_FULL,color,second_color
0,C00371385,N,M2,P,201902059145461441,24K,CCM,THOM TILLIS COMMITTEE,RALEIGH,NC,27624,NaN,NaN,2019-01-09,5000,C00545772,S4NC00162,SB23.6145,1314334,NaN,NaN,4022620191643762260,New,February monthly,Primary,0,Contribution made to nonaffiliated committee,Candidate Committee,SB23,6145,"TILLIS, THOM R. SEN.",I,2,REP,8269271.66,1152018.96,3811302.27,6441.91,2025444.43,6483413.82,0.0,0.0,0.0,0.0,0.0,0.0,4798506.58,NC,00,NaN,NaN,NaN,NaN,NaN,2278244.39,0.0,06/09/2020,102493.33,14425.67,7117252.7,3804860.36,Incumbent,Republican Party,red,pink
1,C00551192,N,M2,P2020,201902119145496265,24K,CCM,THOM TILLIS COMMITTEE,RALEIGH,NC,27624,NaN,NaN,2019-01-28,2700,C00545772,S4NC00162,43079234,1315351,NaN,CONTRIBUTION,4022220191643459655,New,February monthly,Primary,2020,Contribution made to nonaffiliated committee,Candidate Committee,43079234,None,"TILLIS, THOM R. SEN.",I,2,REP,8269271.66,1152018.96,3811302.27,6441.91,2025444.43,6483413.82,0.0,0.0,0.0,0.0,0.0,0.0,4798506.58,NC,00,NaN,NaN,NaN,NaN,NaN,2278244.39,0.0,06/09/2020,102493.33,14425.67,7117252.7,3804860.36,Incumbent,Republican Party,red,pink
2,C00100107,N,M2,P2020,201902159145516138,24K,CCM,THOM TILLIS COMMITTEE,RALEIGH,NC,27624,NaN,NaN,2019-01-11,2500,C00545772,S4NC00162,156BA6A4BB1CA36FD35,1316037,NaN,NaN,4022720191644231227,New,February monthly,Primary,2020,Contribution made to nonaffiliated committee,Candidate Committee,156BA6A4BB1CA36FD35,None,"TILLIS, THOM R. SEN.",I,2,REP,8269271.66,1152018.96,3811302.27,6441.91,2025444.43,6483413.82,0.0,0.0,0.0,0.0,0.0,0.0,4798506.58,NC,00,NaN,NaN,NaN,NaN,NaN,2278244.39,0.0,06/09/2020,102493.33,14425.67,7117252.7,3804860.36,Incumbent,Republican Party,red,pink
3,C00214304,N,M2,G2020,201902199145530707,24K,CCM,THOM TILLIS COMMITTEE,RALEIGH,NC,27624,NaN,NaN,2019-01-08,2500,C00545772,S4NC00162,10001048,1316592,NaN,NaN,4022820191644627257,New,February monthly,General,2020,Contribution made to nonaffiliated committee,Candidate Committee,10001048,None,"TILLIS, THOM R. SEN.",I,2,REP,8269271.66,1152018.96,3811302.27,6441.91,2025444.43,6483413.82,0.0,0.0,0.0,0.0,0.0,0.0,4798506.58,NC,00,NaN,NaN,NaN,NaN,NaN,2278244.39,0.0,06/09/2020,102493.33,14425.67,7117252.7,3804860.36,Incumbent,Republican Party,red,pink
4,C00144774,N,M2,P2020,201902199145526887,24K,CCM,THOM TILLIS COMMITTEE,RALEIGH,NC,27624,NaN,NaN,2019-01-09,5000,C00545772,S4NC00162,12654565,1316428,NaN,DIRECT CONTIRIBUTION,4022720191644226724,New,February monthly,Primary,2020,Contribution made to nonaffiliated committee,Candidate Committee,12654565,None,"TILLIS, THOM R. SEN.",I,2,REP,8269271.66,1152018.96,3811302.27,6441.91,2025444.43,6483413.82,0.0,0.0,0.0,0.0,0.0,0.0,4798506.58,NC,00,NaN,NaN,NaN,NaN,NaN,2278244.39,0.0,06/09/2020,102493.33,14425.67,7117252.7,3804860.36,Incumbent,Republican Party,red,pink


In [27]:
by_entity = committee_contribs.groupby(['CAND_NAME', 'ENTITY_TP_FULL']).agg({'TRANSACTION_AMT': 'sum'}).reset_index()
by_entity_pivot = by_entity.pivot_table(values='TRANSACTION_AMT', index='ENTITY_TP_FULL', columns='CAND_NAME')
by_entity_pivot.iplot(kind='bar', colors=color_map, title='Total Committee contributions & expenditures by entities')

In [28]:
by_transaction = committee_contribs.groupby([
    'CAND_NAME', 'TRANSACTION_TP_FULL']).agg({'TRANSACTION_AMT': 'sum'}).reset_index()

by_transaction_pivot = by_transaction.pivot_table(
    values='TRANSACTION_AMT', index='TRANSACTION_TP_FULL', columns='CAND_NAME')

by_transaction_pivot.iplot(kind='bar', 
                           colors=color_map, title='Total committee contributions & expenditures by transaction type')

In [29]:
committee_master = pd.read_csv(f"../data/20192020-FEC/Committee master.csv")
committee_master.head()

,CMTE_ID,CMTE_NM,TRES_NM,CMTE_ST1,CMTE_ST2,CMTE_CITY,CMTE_ST,CMTE_ZIP,CMTE_DSGN,CMTE_TP,CMTE_PTY_AFFILIATION,CMTE_FILING_FREQ,ORG_TP,CONNECTED_ORG_NM,CAND_ID
0,C00000059,HALLMARK CARDS PAC,SARAH MOE,2501 MCGEE,MD #500,KANSAS CITY,MO,64108,U,Q,UNK,M,C,NaN,NaN
1,C00000422,AMERICAN MEDICAL ASSOCIATION POLITICAL ACTION ...,"WALKER, KEVIN MR.","25 MASSACHUSETTS AVE, NW",SUITE 600,WASHINGTON,DC,200017400,B,Q,NaN,M,M,DELAWARE MEDICAL PAC,NaN
2,C00000489,D R I V E POLITICAL FUND CHAPTER 886,JERRY SIMS JR,3528 W RENO,NaN,OKLAHOMA CITY,OK,73107,U,N,NaN,Q,L,NaN,NaN
3,C00000547,KANSAS MEDICAL SOCIETY POLITICAL ACTION COMMITTEE,JERRY SLAUGHTER,623 SW 10TH AVE,NaN,TOPEKA,KS,666121627,U,Q,UNK,Q,M,KANSAS MEDICAL SOCIETY,NaN
4,C00000638,INDIANA STATE MEDICAL ASSOCIATION POLITICAL AC...,"ACHENBACH, GRANT MR.","322 CANAL WALK, CANAL LEVEL",NaN,INDIANAPOLIS,IN,46202,U,Q,NaN,T,M,NaN,NaN


In [30]:
committee_contribs = committee_contribs.merge(committee_master, on='CMTE_ID', how='left', suffixes=('', '_y'))
committee_contribs.head()

,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,ZIP_CODE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,CAND_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID,AMNDT_IND_FULL,RPT_TP_FULL,ELECTION_TYPE,ELECTION_YEAR,TRANSACTION_TP_FULL,ENTITY_TP_FULL,TRAN_ID_parent,TRAN_ID_child,CAND_NAME,CAND_ICI,PTY_CD,CAND_PTY_AFFILIATION,TTL_RECEIPTS,TRANS_FROM_AUTH,TTL_DISB,TRANS_TO_AUTH,COH_BOP,COH_COP,CAND_CONTRIB,CAND_LOANS,OTHER_LOANS,CAND_LOAN_REPAY,OTHER_LOAN_REPAY,DEBTS_OWED_BY,TTL_INDIV_CONTRIB,CAND_OFFICE_ST,CAND_OFFICE_DISTRICT,SPEC_ELECTION,PRIM_ELECTION,RUN_ELECTION,GEN_ELECTION,GEN_ELECTION_PRECENT,OTHER_POL_CMTE_CONTRIB,POL_PTY_CONTRIB,CVG_END_DT,INDIV_REFUNDS,CMTE_REFUNDS,TTL_RECEIPTS_CORRECTED,TTL_DISB_CORRECTED,CAND_ICI_FULL,CAND_PTY_AFFILIATION_FULL,color,second_color,CMTE_NM,TRES_NM,CMTE_ST1,CMTE_ST2,CMTE_CITY,CMTE_ST,CMTE_ZIP,CMTE_DSGN,CMTE_TP,CMTE_PTY_AFFILIATION,CMTE_FILING_FREQ,ORG_TP,CONNECTED_ORG_NM,CAND_ID_y
0,C00371385,N,M2,P,201902059145461441,24K,CCM,THOM TILLIS COMMITTEE,RALEIGH,NC,27624,NaN,NaN,2019-01-09,5000,C00545772,S4NC00162,SB23.6145,1314334,NaN,NaN,4022620191643762260,New,February monthly,Primary,0,Contribution made to nonaffiliated committee,Candidate Committee,SB23,6145,"TILLIS, THOM R. SEN.",I,2,REP,8269271.66,1152018.96,3811302.27,6441.91,2025444.43,6483413.82,0.0,0.0,0.0,0.0,0.0,0.0,4798506.58,NC,00,NaN,NaN,NaN,NaN,NaN,2278244.39,0.0,06/09/2020,102493.33,14425.67,7117252.7,3804860.36,Incumbent,Republican Party,red,pink,ALEX LEE INC PAC,"ALMQUIST, ANDREW MR.",120 4TH STREET SW,NaN,HICKORY,NC,28602,U,Q,NaN,M,C,"ALEX LEE, INC.",NaN
1,C00551192,N,M2,P2020,201902119145496265,24K,CCM,THOM TILLIS COMMITTEE,RALEIGH,NC,27624,NaN,NaN,2019-01-28,2700,C00545772,S4NC00162,43079234,1315351,NaN,CONTRIBUTION,4022220191643459655,New,February monthly,Primary,2020,Contribution made to nonaffiliated committee,Candidate Committee,43079234,None,"TILLIS, THOM R. SEN.",I,2,REP,8269271.66,1152018.96,3811302.27,6441.91,2025444.43,6483413.82,0.0,0.0,0.0,0.0,0.0,0.0,4798506.58,NC,00,NaN,NaN,NaN,NaN,NaN,2278244.39,0.0,06/09/2020,102493.33,14425.67,7117252.7,3804860.36,Incumbent,Republican Party,red,pink,MAIN STREET BANKING POLITICAL ACTION COMMITTEE,"WILLIAMS, WADE",PO BOX 7427,NaN,ALEXANDRIA,VA,223077427,U,N,NaN,M,NaN,NONE,NaN
2,C00100107,N,M2,P2020,201902159145516138,24K,CCM,THOM TILLIS COMMITTEE,RALEIGH,NC,27624,NaN,NaN,2019-01-11,2500,C00545772,S4NC00162,156BA6A4BB1CA36FD35,1316037,NaN,NaN,4022720191644231227,New,February monthly,Primary,2020,Contribution made to nonaffiliated committee,Candidate Committee,156BA6A4BB1CA36FD35,None,"TILLIS, THOM R. SEN.",I,2,REP,8269271.66,1152018.96,3811302.27,6441.91,2025444.43,6483413.82,0.0,0.0,0.0,0.0,0.0,0.0,4798506.58,NC,00,NaN,NaN,NaN,NaN,NaN,2278244.39,0.0,06/09/2020,102493.33,14425.67,7117252.7,3804860.36,Incumbent,Republican Party,red,pink,AMERICAN BEVERAGE ASSOCIATION POLITICAL ACTION...,"HAMMOND, MARK N. MR.",1275 PENNSYLVANIA AVE NW,SUITE 1100,WASHINGTON,DC,20004,B,Q,NNE,M,T,AMERICAN BEVERAGE ASSOCIATION,NaN
3,C00214304,N,M2,G2020,201902199145530707,24K,CCM,THOM TILLIS COMMITTEE,RALEIGH,NC,27624,NaN,NaN,2019-01-08,2500,C00545772,S4NC00162,10001048,1316592,NaN,NaN,4022820191644627257,New,February monthly,General,2020,Contribution made to nonaffiliated committee,Candidate Committee,10001048,None,"TILLIS, THOM R. SEN.",I,2,REP,8269271.66,1152018.96,3811302.27,6441.91,2025444.43,6483413.82,0.0,0.0,0.0,0.0,0.0,0.0,4798506.58,NC,00,NaN,NaN,NaN,NaN,NaN,2278244.39,0.0,06/09/2020,102493.33,14425.67,7117252.7,3804860.36,Incumbent,Republican Party,red,pink,"AHOLD DELHAIZE USA, INC POLITICAL ACTION COMMI...","YOUNG, TEROSS WILLIEVICK",PO BOX 1330,2110 EXECUTIVE DRIVE,SALISBURY,NC,28145,U,Q,NaN,M,C,"AHOLD DELHAIZE USA, INC.",NaN
4,C00144774,N,M2,P2020,201902199145526887,24K,CCM,THOM TILLIS COMMITTEE,RALEIGH,NC,27624,NaN,NaN,2019-01-09,5000,C00545772,S4NC00162,12654565,1316428,NaN,DIRECT CONTIRIBUTION,4022720191644226724,New,February mont

### Contributions from committees

In [31]:
contribs = committee_contribs[committee_contribs['TRANSACTION_TP'].isin(['24K', '24Z'])]
contribs.head()

,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,ZIP_CODE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,CAND_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID,AMNDT_IND_FULL,RPT_TP_FULL,ELECTION_TYPE,ELECTION_YEAR,TRANSACTION_TP_FULL,ENTITY_TP_FULL,TRAN_ID_parent,TRAN_ID_child,CAND_NAME,CAND_ICI,PTY_CD,CAND_PTY_AFFILIATION,TTL_RECEIPTS,TRANS_FROM_AUTH,TTL_DISB,TRANS_TO_AUTH,COH_BOP,COH_COP,CAND_CONTRIB,CAND_LOANS,OTHER_LOANS,CAND_LOAN_REPAY,OTHER_LOAN_REPAY,DEBTS_OWED_BY,TTL_INDIV_CONTRIB,CAND_OFFICE_ST,CAND_OFFICE_DISTRICT,SPEC_ELECTION,PRIM_ELECTION,RUN_ELECTION,GEN_ELECTION,GEN_ELECTION_PRECENT,OTHER_POL_CMTE_CONTRIB,POL_PTY_CONTRIB,CVG_END_DT,INDIV_REFUNDS,CMTE_REFUNDS,TTL_RECEIPTS_CORRECTED,TTL_DISB_CORRECTED,CAND_ICI_FULL,CAND_PTY_AFFILIATION_FULL,color,second_color,CMTE_NM,TRES_NM,CMTE_ST1,CMTE_ST2,CMTE_CITY,CMTE_ST,CMTE_ZIP,CMTE_DSGN,CMTE_TP,CMTE_PTY_AFFILIATION,CMTE_FILING_FREQ,ORG_TP,CONNECTED_ORG_NM,CAND_ID_y
0,C00371385,N,M2,P,201902059145461441,24K,CCM,THOM TILLIS COMMITTEE,RALEIGH,NC,27624,NaN,NaN,2019-01-09,5000,C00545772,S4NC00162,SB23.6145,1314334,NaN,NaN,4022620191643762260,New,February monthly,Primary,0,Contribution made to nonaffiliated committee,Candidate Committee,SB23,6145,"TILLIS, THOM R. SEN.",I,2,REP,8269271.66,1152018.96,3811302.27,6441.91,2025444.43,6483413.82,0.0,0.0,0.0,0.0,0.0,0.0,4798506.58,NC,00,NaN,NaN,NaN,NaN,NaN,2278244.39,0.0,06/09/2020,102493.33,14425.67,7117252.7,3804860.36,Incumbent,Republican Party,red,pink,ALEX LEE INC PAC,"ALMQUIST, ANDREW MR.",120 4TH STREET SW,NaN,HICKORY,NC,28602,U,Q,NaN,M,C,"ALEX LEE, INC.",NaN
1,C00551192,N,M2,P2020,201902119145496265,24K,CCM,THOM TILLIS COMMITTEE,RALEIGH,NC,27624,NaN,NaN,2019-01-28,2700,C00545772,S4NC00162,43079234,1315351,NaN,CONTRIBUTION,4022220191643459655,New,February monthly,Primary,2020,Contribution made to nonaffiliated committee,Candidate Committee,43079234,None,"TILLIS, THOM R. SEN.",I,2,REP,8269271.66,1152018.96,3811302.27,6441.91,2025444.43,6483413.82,0.0,0.0,0.0,0.0,0.0,0.0,4798506.58,NC,00,NaN,NaN,NaN,NaN,NaN,2278244.39,0.0,06/09/2020,102493.33,14425.67,7117252.7,3804860.36,Incumbent,Republican Party,red,pink,MAIN STREET BANKING POLITICAL ACTION COMMITTEE,"WILLIAMS, WADE",PO BOX 7427,NaN,ALEXANDRIA,VA,223077427,U,N,NaN,M,NaN,NONE,NaN
2,C00100107,N,M2,P2020,201902159145516138,24K,CCM,THOM TILLIS COMMITTEE,RALEIGH,NC,27624,NaN,NaN,2019-01-11,2500,C00545772,S4NC00162,156BA6A4BB1CA36FD35,1316037,NaN,NaN,4022720191644231227,New,February monthly,Primary,2020,Contribution made to nonaffiliated committee,Candidate Committee,156BA6A4BB1CA36FD35,None,"TILLIS, THOM R. SEN.",I,2,REP,8269271.66,1152018.96,3811302.27,6441.91,2025444.43,6483413.82,0.0,0.0,0.0,0.0,0.0,0.0,4798506.58,NC,00,NaN,NaN,NaN,NaN,NaN,2278244.39,0.0,06/09/2020,102493.33,14425.67,7117252.7,3804860.36,Incumbent,Republican Party,red,pink,AMERICAN BEVERAGE ASSOCIATION POLITICAL ACTION...,"HAMMOND, MARK N. MR.",1275 PENNSYLVANIA AVE NW,SUITE 1100,WASHINGTON,DC,20004,B,Q,NNE,M,T,AMERICAN BEVERAGE ASSOCIATION,NaN
3,C00214304,N,M2,G2020,201902199145530707,24K,CCM,THOM TILLIS COMMITTEE,RALEIGH,NC,27624,NaN,NaN,2019-01-08,2500,C00545772,S4NC00162,10001048,1316592,NaN,NaN,4022820191644627257,New,February monthly,General,2020,Contribution made to nonaffiliated committee,Candidate Committee,10001048,None,"TILLIS, THOM R. SEN.",I,2,REP,8269271.66,1152018.96,3811302.27,6441.91,2025444.43,6483413.82,0.0,0.0,0.0,0.0,0.0,0.0,4798506.58,NC,00,NaN,NaN,NaN,NaN,NaN,2278244.39,0.0,06/09/2020,102493.33,14425.67,7117252.7,3804860.36,Incumbent,Republican Party,red,pink,"AHOLD DELHAIZE USA, INC POLITICAL ACTION COMMI...","YOUNG, TEROSS WILLIEVICK",PO BOX 1330,2110 EXECUTIVE DRIVE,SALISBURY,NC,28145,U,Q,NaN,M,C,"AHOLD DELHAIZE USA, INC.",NaN
4,C00144774,N,M2,P2020,201902199145526887,24K,CCM,THOM TILLIS COMMITTEE,RALEIGH,NC,27624,NaN,NaN,2019-01-09,5000,C00545772,S4NC00162,12654565,1316428,NaN,DIRECT CONTIRIBUTION,4022720191644226724,New,February mont

#### Here are the committees with the largest contributions

In [32]:
largest_contribs = contribs.groupby('CMTE_NM').agg(
    {'TRANSACTION_AMT': 'sum'})['TRANSACTION_AMT'].nlargest(10).reset_index()

largest_contribs['TRANSACTION_AMT'] = largest_contribs['TRANSACTION_AMT'].map(lambda x: render_human_format(x))

largest_contribs.rename({'CMTE_NM': 'COMMITTEE_NAME'}, axis=1)

,COMMITTEE_NAME,TRANSACTION_AMT
0,DSCC,49.60K
1,NRSC,44.60K
2,JSTREETPAC,34.93K
3,ABBOTT LABORATORIES EMPLOYEE POLITICAL ACTION ...,10.00K
4,ALL FOR OUR COUNTRY LEADERSHIP PAC,10.00K
5,"ALLERGAN, INC. POLITICAL ACTION COMMITTEE",10.00K
6,AMERICAN BANKERS ASSOCIATION PAC (BANKPAC),10.00K
7,ANADARKO PETROLEUM CORPORATION POLITICAL ACTIO...,10.00K
8,ARKANSAS FOR LEADERSHIP POLITICAL ACTION COMMI...,10.00K
9,"ASSOCIATED BUILDERS AND CONTRACTORS, INC. POLI...",10.00K


In [33]:
from IPython.display import display
for cand_name in candidates['CAND_NAME'].unique():
    print("-"*60)
    print(f"Committees with most contributions for {cand_name}")
    print("-"*60)
    df = contribs[(contribs['CAND_NAME'] == cand_name)]
    result = pd.DataFrame(df[['CMTE_NM', 'TRANSACTION_AMT']].groupby(['CMTE_NM']).agg({'TRANSACTION_AMT': 'sum'})['TRANSACTION_AMT'].nlargest(10)).reset_index().rename({'CMTE_NM': 'COMMITTEE_NAME'}, axis=1)
    result['TRANSACTION_AMT'] = result['TRANSACTION_AMT'].map(lambda x: render_human_format(x))
    display(result)

------------------------------------------------------------
Committees with most contributions for CUNNINGHAM, CAL
------------------------------------------------------------


,COMMITTEE_NAME,TRANSACTION_AMT
0,DSCC,49.60K
1,JSTREETPAC,34.93K
2,ALL FOR OUR COUNTRY LEADERSHIP PAC,10.00K
3,COMMON GROUND PAC,10.00K
4,COMMUNICATIONS WORKERS OF AMERICA-COPE POLITIC...,10.00K
5,FIRST STATE PAC,10.00K
6,FORWARD TOGETHER PAC,10.00K
7,GETTING STUFF DONE PAC (GSD-PAC),10.00K
8,HAWAII PAC,10.00K
9,HOOPS PAC,10.00K


------------------------------------------------------------
Committees with most contributions for TILLIS, THOM R. SEN.
------------------------------------------------------------


,COMMITTEE_NAME,TRANSACTION_AMT
0,NRSC,44.60K
1,ABBOTT LABORATORIES EMPLOYEE POLITICAL ACTION ...,10.00K
2,"ALLERGAN, INC. POLITICAL ACTION COMMITTEE",10.00K
3,AMERICAN BANKERS ASSOCIATION PAC (BANKPAC),10.00K
4,ANADARKO PETROLEUM CORPORATION POLITICAL ACTIO...,10.00K
5,ARKANSAS FOR LEADERSHIP POLITICAL ACTION COMMI...,10.00K
6,"ASSOCIATED BUILDERS AND CONTRACTORS, INC. POLI...",10.00K
7,ASSOCIATION OF KENTUCKY FRIED CHICKEN FRANCHIS...,10.00K
8,BANK POLICY INSTITUTE PAC,10.00K
9,"BBVA USA BANCSHARES, INC. PAC",10.00K


In [34]:
ctrbs_by_dt = contribs.groupby(['TRANSACTION_DT', 
                                'CAND_NAME']).agg({'TRANSACTION_AMT': 'sum'}).reset_index()
ctrbs_by_dt = ctrbs_by_dt.pivot_table(index='TRANSACTION_DT', 
                                      columns=['CAND_NAME'], fill_value=0.0).droplevel(level=0, axis=1)
ctrbs_by_dt.cumsum().iplot(title='Total cumulative Committee Contributions for candidates over time', 
                           colors=color_map)

### Committee expenditures for candidates

In [35]:
committee_contribs['for_against'] = committee_contribs['TRANSACTION_TP'].map({
    '24A': 'Against', '24E': 'For'})

expenditures = committee_contribs[~committee_contribs['for_against'].isnull()]
expenditures.head()

,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,ZIP_CODE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,CAND_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID,AMNDT_IND_FULL,RPT_TP_FULL,ELECTION_TYPE,ELECTION_YEAR,TRANSACTION_TP_FULL,ENTITY_TP_FULL,TRAN_ID_parent,TRAN_ID_child,CAND_NAME,CAND_ICI,PTY_CD,CAND_PTY_AFFILIATION,TTL_RECEIPTS,TRANS_FROM_AUTH,TTL_DISB,TRANS_TO_AUTH,COH_BOP,COH_COP,CAND_CONTRIB,CAND_LOANS,OTHER_LOANS,CAND_LOAN_REPAY,OTHER_LOAN_REPAY,DEBTS_OWED_BY,TTL_INDIV_CONTRIB,CAND_OFFICE_ST,CAND_OFFICE_DISTRICT,SPEC_ELECTION,PRIM_ELECTION,RUN_ELECTION,GEN_ELECTION,GEN_ELECTION_PRECENT,OTHER_POL_CMTE_CONTRIB,POL_PTY_CONTRIB,CVG_END_DT,INDIV_REFUNDS,CMTE_REFUNDS,TTL_RECEIPTS_CORRECTED,TTL_DISB_CORRECTED,CAND_ICI_FULL,CAND_PTY_AFFILIATION_FULL,color,second_color,CMTE_NM,TRES_NM,CMTE_ST1,CMTE_ST2,CMTE_CITY,CMTE_ST,CMTE_ZIP,CMTE_DSGN,CMTE_TP,CMTE_PTY_AFFILIATION,CMTE_FILING_FREQ,ORG_TP,CONNECTED_ORG_NM,CAND_ID_y,for_against
494,C00571703,N,YE,P2020,202001319185087439,24E,ORG,MCCARTHY HENNINGS WHALEN INC.,WASHINGTON,DC,20036,NaN,NaN,2019-12-20,2734,S4NC00162,S4NC00162,SE24.15378,1378798,NaN,NaN,4020120201686471398,New,Year end,Primary,2020,Independent expenditure advocating election of...,Organization (not a committee and not a person),SE24,15378,"TILLIS, THOM R. SEN.",I,2,REP,8269271.66,1152018.96,3811302.27,6441.91,2025444.43,6483413.82,0.0,0.0,0.0,0.0,0.0,0.0,4798506.58,NC,00,NaN,NaN,NaN,NaN,NaN,2278244.39,0.0,06/09/2020,102493.33,14425.67,7117252.7,3804860.36,Incumbent,Republican Party,red,pink,SENATE LEADERSHIP FUND,CALEB CROSBY,45 NORTH HILL DRIVE STE 100,NaN,WARRENTON,VA,20186,U,O,NaN,M,NaN,NaN,NaN,For
495,C00571703,N,YE,P2020,202001319185087444,24E,ORG,RICHARD SALES MEDIA,PHOENIX,AZ,85016,NaN,NaN,2019-12-20,600,S4NC00162,S4NC00162,SE24.15376,1378798,NaN,NaN,4020120201686471408,New,Year end,Primary,2020,Independent expenditure advocating election of...,Organization (not a committee and not a person),SE24,15376,"TILLIS, THOM R. SEN.",I,2,REP,8269271.66,1152018.96,3811302.27,6441.91,2025444.43,6483413.82,0.0,0.0,0.0,0.0,0.0,0.0,4798506.58,NC,00,NaN,NaN,NaN,NaN,NaN,2278244.39,0.0,06/09/2020,102493.33,14425.67,7117252.7,3804860.36,Incumbent,Republican Party,red,pink,SENATE LEADERSHIP FUND,CALEB CROSBY,45 NORTH HILL DRIVE STE 100,NaN,WARRENTON,VA,20186,U,O,NaN,M,NaN,NaN,NaN,For
496,C00571703,N,YE,P2020,202001319185087444,24E,ORG,RICHARD SALES MEDIA,PHOENIX,AZ,85016,NaN,NaN,2019-12-20,400,S4NC00162,S4NC00162,SE24.15377,1378798,NaN,NaN,4020120201686471409,New,Year end,Primary,2020,Independent expenditure advocating election of...,Organization (not a committee and not a person),SE24,15377,"TILLIS, THOM R. SEN.",I,2,REP,8269271.66,1152018.96,3811302.27,6441.91,2025444.43,6483413.82,0.0,0.0,0.0,0.0,0.0,0.0,4798506.58,NC,00,NaN,NaN,NaN,NaN,NaN,2278244.39,0.0,06/09/2020,102493.33,14425.67,7117252.7,3804860.36,Incumbent,Republican Party,red,pink,SENATE LEADERSHIP FUND,CALEB CROSBY,45 NORTH HILL DRIVE STE 100,NaN,WARRENTON,VA,20186,U,O,NaN,M,NaN,NaN,NaN,For
497,C00571703,N,YE,P2020,202001319185087445,24E,ORG,TARGETED VICTORY LLC,ARLINGTON,VA,22201,NaN,NaN,2019-12-11,50000,S4NC00162,S4NC00162,SE24.15375,1378798,NaN,NaN,4020120201686471410,New,Year end,Primary,2020,Independent expenditure advocating election of...,Organization (not a committee and not a person),SE24,15375,"TILLIS, THOM R. SEN.",I,2,REP,8269271.66,1152018.96,3811302.27,6441.91,2025444.43,6483413.82,0.0,0.0,0.0,0.0,0.0,0.0,4798506.58,NC,00,NaN,NaN,NaN,NaN,NaN,2278244.39,0.0,06/09/2020,102493.33,14425.67,7117252.7,3804860.36,Incumbent,Republican Party,red,pink,SENATE LEADERSHIP FUND,CALEB CROSBY,45 NORTH HILL DRIVE STE 100,NaN,WARRENTON,VA,20186,U,O,NaN,M,NaN,NaN,NaN,For
539,C00514125,N,YE,P2020,202001299182384741,24A,ORG,"UP ON AIR BUYING, LLC",DALLAS,TX,75219,NaN,NaN,2019-10-11,254000,S4NC00162,S4NC00162,SE.4447,1374809,NaN,NaN,4013020201686174178,New,Year end,Primary,2020,Independent expenditure opposing elect

**Here are the committees with the largest expenditures**

In [36]:
largest_expenditures = expenditures.groupby('CMTE_NM').agg({'TRANSACTION_AMT': 'sum'})['TRANSACTION_AMT'].nlargest(10).reset_index()
largest_expenditures['TRANSACTION_AMT'] = largest_expenditures['TRANSACTION_AMT'].map(lambda x: render_human_format(x))
largest_expenditures.rename({'CMTE_NM': 'COMMITTEE_NAME'}, axis=1)

,COMMITTEE_NAME,TRANSACTION_AMT
0,CAROLINA BLUE,4.52M
1,VOTEVETS,4.29M
2,"AMERICANS FOR PROSPERITY ACTION, INC.(AFP ACTION)",675.66K
3,THE AMERICAN FOUNDATIONS COMMITTEE,254.00K
4,FAITH AND POWER PAC,250.01K
5,SENATE LEADERSHIP FUND,66.99K
6,INDIVISIBLE ACTION,22.50K
7,THE LINCOLN PROJECT,11.78K
8,NATIONAL RIFLE ASSOCIATION OF AMERICA POLITICA...,10.07K
9,HEALTH JUSTICE FOR ALL,6.52K


In [37]:
from IPython.display import display
for cand_name in candidates['CAND_NAME'].unique():
    for for_against in ['For', 'Against']:
        print("-"*60)
        print(f"Committees with most expenditures {for_against} {cand_name}")
        print("-"*60)
        df = expenditures[(expenditures['for_against'] == for_against) & (expenditures['CAND_NAME'] == cand_name)]
        result = pd.DataFrame(df[['CMTE_NM', 'TRANSACTION_AMT']].groupby(['CMTE_NM']).agg({'TRANSACTION_AMT': 'sum'})['TRANSACTION_AMT'].nlargest(10)).reset_index().rename({'CMTE_NM': 'COMMITTEE_NAME'}, axis=1)
        result['TRANSACTION_AMT'] = result['TRANSACTION_AMT'].map(lambda x: render_human_format(x))
        display(result)

------------------------------------------------------------
Committees with most expenditures For CUNNINGHAM, CAL
------------------------------------------------------------


,COMMITTEE_NAME,TRANSACTION_AMT
0,CAROLINA BLUE,4.52M
1,VOTEVETS,4.29M
2,PAGE COMMUNICATIONS L.L.C.,3.26K
3,PLANNED PARENTHOOD VOTES,579.00
4,SIERRA CLUB INDEPENDENT ACTION,10.00


------------------------------------------------------------
Committees with most expenditures Against CUNNINGHAM, CAL
------------------------------------------------------------


,COMMITTEE_NAME,TRANSACTION_AMT
0,FAITH AND POWER PAC,250.01K


------------------------------------------------------------
Committees with most expenditures For TILLIS, THOM R. SEN.
------------------------------------------------------------


,COMMITTEE_NAME,TRANSACTION_AMT
0,"AMERICANS FOR PROSPERITY ACTION, INC.(AFP ACTION)",675.66K
1,SENATE LEADERSHIP FUND,66.99K
2,NATIONAL RIFLE ASSOCIATION OF AMERICA POLITICA...,10.07K
3,WOMEN SPEAK OUT PAC,4.54K


------------------------------------------------------------
Committees with most expenditures Against TILLIS, THOM R. SEN.
------------------------------------------------------------


,COMMITTEE_NAME,TRANSACTION_AMT
0,THE AMERICAN FOUNDATIONS COMMITTEE,254.00K
1,INDIVISIBLE ACTION,22.50K
2,THE LINCOLN PROJECT,11.78K
3,HEALTH JUSTICE FOR ALL,6.52K
4,PLANNED PARENTHOOD VOTES,866.00


In [38]:
exp_by_dt = expenditures.groupby(['TRANSACTION_DT', 'CAND_NAME', 'for_against']).agg({'TRANSACTION_AMT': 'sum'}).reset_index()
exp_by_dt = exp_by_dt.pivot_table(index='TRANSACTION_DT', columns=['CAND_NAME', 'for_against'], fill_value=0.0).droplevel(level=0, axis=1)
exp_by_dt.columns = exp_by_dt.columns.map(lambda x: x[1] + " " + x[0])
exp_by_dt.cumsum().iplot(title='Total cumulative Expenditures for or against candidates over time', colors=secondary_color_map)

## Contributions from individuals

In [39]:
# First, we determine which committees are for or against a candidate so that we can bucket the contribution appropriately
committee_contribs['for_against'] = committee_contribs['for_against'].fillna('For')
committee_contribs['CAND_NAME'] = np.where(((committee_contribs['for_against'] == 'Against') & 
                                            (committee_contribs['CAND_NAME'] == democrat)) |
                                            ((committee_contribs['for_against'] == 'For') & 
                                            (committee_contribs['CAND_NAME'] == republican)),
                                           republican, democrat)
committee_contribs.head()

,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,ZIP_CODE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,CAND_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID,AMNDT_IND_FULL,RPT_TP_FULL,ELECTION_TYPE,ELECTION_YEAR,TRANSACTION_TP_FULL,ENTITY_TP_FULL,TRAN_ID_parent,TRAN_ID_child,CAND_NAME,CAND_ICI,PTY_CD,CAND_PTY_AFFILIATION,TTL_RECEIPTS,TRANS_FROM_AUTH,TTL_DISB,TRANS_TO_AUTH,COH_BOP,COH_COP,CAND_CONTRIB,CAND_LOANS,OTHER_LOANS,CAND_LOAN_REPAY,OTHER_LOAN_REPAY,DEBTS_OWED_BY,TTL_INDIV_CONTRIB,CAND_OFFICE_ST,CAND_OFFICE_DISTRICT,SPEC_ELECTION,PRIM_ELECTION,RUN_ELECTION,GEN_ELECTION,GEN_ELECTION_PRECENT,OTHER_POL_CMTE_CONTRIB,POL_PTY_CONTRIB,CVG_END_DT,INDIV_REFUNDS,CMTE_REFUNDS,TTL_RECEIPTS_CORRECTED,TTL_DISB_CORRECTED,CAND_ICI_FULL,CAND_PTY_AFFILIATION_FULL,color,second_color,CMTE_NM,TRES_NM,CMTE_ST1,CMTE_ST2,CMTE_CITY,CMTE_ST,CMTE_ZIP,CMTE_DSGN,CMTE_TP,CMTE_PTY_AFFILIATION,CMTE_FILING_FREQ,ORG_TP,CONNECTED_ORG_NM,CAND_ID_y,for_against
0,C00371385,N,M2,P,201902059145461441,24K,CCM,THOM TILLIS COMMITTEE,RALEIGH,NC,27624,NaN,NaN,2019-01-09,5000,C00545772,S4NC00162,SB23.6145,1314334,NaN,NaN,4022620191643762260,New,February monthly,Primary,0,Contribution made to nonaffiliated committee,Candidate Committee,SB23,6145,"TILLIS, THOM R. SEN.",I,2,REP,8269271.66,1152018.96,3811302.27,6441.91,2025444.43,6483413.82,0.0,0.0,0.0,0.0,0.0,0.0,4798506.58,NC,00,NaN,NaN,NaN,NaN,NaN,2278244.39,0.0,06/09/2020,102493.33,14425.67,7117252.7,3804860.36,Incumbent,Republican Party,red,pink,ALEX LEE INC PAC,"ALMQUIST, ANDREW MR.",120 4TH STREET SW,NaN,HICKORY,NC,28602,U,Q,NaN,M,C,"ALEX LEE, INC.",NaN,For
1,C00551192,N,M2,P2020,201902119145496265,24K,CCM,THOM TILLIS COMMITTEE,RALEIGH,NC,27624,NaN,NaN,2019-01-28,2700,C00545772,S4NC00162,43079234,1315351,NaN,CONTRIBUTION,4022220191643459655,New,February monthly,Primary,2020,Contribution made to nonaffiliated committee,Candidate Committee,43079234,None,"TILLIS, THOM R. SEN.",I,2,REP,8269271.66,1152018.96,3811302.27,6441.91,2025444.43,6483413.82,0.0,0.0,0.0,0.0,0.0,0.0,4798506.58,NC,00,NaN,NaN,NaN,NaN,NaN,2278244.39,0.0,06/09/2020,102493.33,14425.67,7117252.7,3804860.36,Incumbent,Republican Party,red,pink,MAIN STREET BANKING POLITICAL ACTION COMMITTEE,"WILLIAMS, WADE",PO BOX 7427,NaN,ALEXANDRIA,VA,223077427,U,N,NaN,M,NaN,NONE,NaN,For
2,C00100107,N,M2,P2020,201902159145516138,24K,CCM,THOM TILLIS COMMITTEE,RALEIGH,NC,27624,NaN,NaN,2019-01-11,2500,C00545772,S4NC00162,156BA6A4BB1CA36FD35,1316037,NaN,NaN,4022720191644231227,New,February monthly,Primary,2020,Contribution made to nonaffiliated committee,Candidate Committee,156BA6A4BB1CA36FD35,None,"TILLIS, THOM R. SEN.",I,2,REP,8269271.66,1152018.96,3811302.27,6441.91,2025444.43,6483413.82,0.0,0.0,0.0,0.0,0.0,0.0,4798506.58,NC,00,NaN,NaN,NaN,NaN,NaN,2278244.39,0.0,06/09/2020,102493.33,14425.67,7117252.7,3804860.36,Incumbent,Republican Party,red,pink,AMERICAN BEVERAGE ASSOCIATION POLITICAL ACTION...,"HAMMOND, MARK N. MR.",1275 PENNSYLVANIA AVE NW,SUITE 1100,WASHINGTON,DC,20004,B,Q,NNE,M,T,AMERICAN BEVERAGE ASSOCIATION,NaN,For
3,C00214304,N,M2,G2020,201902199145530707,24K,CCM,THOM TILLIS COMMITTEE,RALEIGH,NC,27624,NaN,NaN,2019-01-08,2500,C00545772,S4NC00162,10001048,1316592,NaN,NaN,4022820191644627257,New,February monthly,General,2020,Contribution made to nonaffiliated committee,Candidate Committee,10001048,None,"TILLIS, THOM R. SEN.",I,2,REP,8269271.66,1152018.96,3811302.27,6441.91,2025444.43,6483413.82,0.0,0.0,0.0,0.0,0.0,0.0,4798506.58,NC,00,NaN,NaN,NaN,NaN,NaN,2278244.39,0.0,06/09/2020,102493.33,14425.67,7117252.7,3804860.36,Incumbent,Republican Party,red,pink,"AHOLD DELHAIZE USA, INC POLITICAL ACTION COMMI...","YOUNG, TEROSS WILLIEVICK",PO BOX 1330,2110 EXECUTIVE DRIVE,SALISBURY,NC,28145,U,Q,NaN,M,C,"AHOLD DELHAIZE USA, INC.",NaN,For
4,C00144774,N,M2,P2020,201902199145526887,24K,CCM,THOM TILLIS COMMITTEE,RALEIGH,NC,27624,NaN,NaN,2019-01-09,5000,C00545772,S4NC00162,12654565,1316428,NaN,DIRECT CONTIRIBUTION,402272019

In [40]:
cmte_cand_mapping = committee_contribs.groupby(['CMTE_ID', 'CAND_NAME']).agg({'TRANSACTION_AMT': 'sum'})
cmte_cand_mapping = cmte_cand_mapping.pivot_table(index='CMTE_ID', columns='CAND_NAME').fillna(0.0).droplevel(0, axis=1).reset_index()
cmte_cand_mapping = cmte_cand_mapping.loc[~(cmte_cand_mapping[democrat] == cmte_cand_mapping[republican])]
cmte_cand_mapping['CAND_NAME'] = np.where(cmte_cand_mapping[democrat] > cmte_cand_mapping[republican], democrat, republican)
cmte_cand_mapping.head()

CAND_NAME,CMTE_ID,"CUNNINGHAM, CAL","TILLIS, THOM R. SEN.",CAND_NAME
0,C00000422,0.0,2500.0,"TILLIS, THOM R. SEN."
1,C00000729,0.0,3000.0,"TILLIS, THOM R. SEN."
2,C00000885,2500.0,0.0,"CUNNINGHAM, CAL"
3,C00000901,0.0,5000.0,"TILLIS, THOM R. SEN."
4,C00001016,5000.0,0.0,"CUNNINGHAM, CAL"


In [41]:
s = f"../data/20192020-FEC/Contributions by individuals.csv"
cmd_result = ! wc -l "{s}"
contribs_cnt = int(cmd_result[0].split()[0])
contribs_cnt

26170487

In [42]:
# Iteration 42 has some bad data that needs to be fixed manually as below
def handle_iteration_42(df):
    bad_row = df[df['TRANSACTION_DT'] == 'SAN DIEGO']
    bad_rows = bad_row['OCCUPATION'].str.split('\n').values[0]
    other_rows = []
    for i, row in enumerate(bad_rows):
        if i==0:
            first_row_last_part = row.split('|')
        elif i == len(bad_rows)-1:
            last_row_first_part = row.split('|')
        else:
            other_rows.append(row.split('|'))
    all_dfs = [df]
    first_row = list(df.loc[bad_row.index].iloc[:, 0:12].copy().values[0]) + first_row_last_part
    all_dfs.append(pd.DataFrame([first_row], columns=df.columns))
    all_dfs.append(pd.DataFrame(other_rows, columns=df.columns))
    all_dfs.append(pd.DataFrame([last_row_first_part + list(df.loc[bad_row.index].iloc[:, 13:].copy().values[0])
                                   + [np.NaN, np.NaN, np.NaN, np.NaN, np.NaN]], columns=df.columns))
    df = pd.concat(all_dfs, axis=0)
    df = df.drop(index=bad_row.index).reset_index()
    return df

In [44]:
import gc
chunksize = 100000
contribs_by_indivs = []
cols = None
for i in range(contribs_cnt//chunksize):
    print(f"Processing chunk {i}")
    df = pd.read_csv(f"../data/20192020-FEC/Contributions by individuals.csv", skiprows=i*chunksize, nrows=chunksize)
    if i == 0:
        cols = df.columns
    else:
        df.columns = cols
    if i == 42:
        df = handle_iteration_42(df)
    interesting_ones = df[df['CMTE_ID'].isin(cmte_cand_mapping['CMTE_ID'])]
    if len(interesting_ones) > 0:
        contribs_by_indivs.append(interesting_ones)
#len(contribs_by_indivs)

Processing chunk 0


/Users/sahluwalia/acre/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3257: DtypeWarning:

Columns (10,18) have mixed types.Specify dtype option on import or set low_memory=False.



Processing chunk 1


/Users/sahluwalia/acre/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3257: DtypeWarning:

Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.



Processing chunk 2
Processing chunk 3
Processing chunk 4
Processing chunk 5
Processing chunk 6


...

Processing chunk 254
Processing chunk 255
Processing chunk 256


/Users/sahluwalia/acre/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3257: DtypeWarning:

Columns (11,12) have mixed types.Specify dtype option on import or set low_memory=False.



Processing chunk 257


/Users/sahluwalia/acre/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3257: DtypeWarning:

Columns (3,10,11,12,19) have mixed types.Specify dtype option on import or set low_memory=False.



Processing chunk 258
Processing chunk 259
Processing chunk 260


In [45]:
contribs_by_indivs = pd.concat(contribs_by_indivs)
contribs_by_indivs.head()

,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,ZIP_CODE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID
15,C00428110,N,M2,P,201902049145460549,15,IND,"MATTHIES, KEVIN",WICHITA,KS,67210,SPIRIT AEROSYSTEMS INC,SENIOR VP/GENERAL MANAGER,1102019.0,208,NaN,B000235S000077L11A1,1314300,NaN,NaN,4022620191644153387
16,C00428110,N,M2,P,201902049145460548,15,IND,"GENTILE, THOMAS C",WICHITA,KS,67210,SPIRIT AEROSYSTEMS INC,PRESIDENT & CEO,1102019.0,416,NaN,B000235S000036L11A1,1314300,NaN,NaN,4022620191644153384
17,C00428110,N,M2,P,201902049145460548,15,IND,"HAWKINS, DUANE F",WICHITA,KS,67210,SPIRIT AEROSYSTEMS INC,SENIOR VP/GENERAL MANAGER,1102019.0,250,NaN,B000235S000045L11A1,1314300,NaN,NaN,4022620191644153385
18,C00428110,N,M2,P,201902049145460548,15,IND,"KONDROTIS, KRISSTIE A",WICHITA,KS,67210,SPIRIT AEROSYSTEMS INC,SVP BUSINESS DEVELOPMENT & REGIO,1102019.0,250,NaN,B000235S000062L11A1,1314300,NaN,NaN,4022620191644153386
19,C00428110,N,M2,P,201902049145460549,15,IND,"PILLA, JOHN ANTHONY",WICHITA,KS,67210,SPIRIT AEROSYSTEMS INC,SVP ENGINEERING AND CTO,1102019.0,250,NaN,B000235S000090L11A1,1314300,NaN,NaN,4022620191644153388


In [46]:
contribs_by_indivs = contribs_by_indivs[(~contribs_by_indivs['TRANSACTION_DT'].isnull())]
contribs_by_indivs['TRANSACTION_DT'] = pd.to_datetime(
    contribs_by_indivs['TRANSACTION_DT'].fillna(0).astype('int').astype('str')
    .replace('0', np.NaN).str.zfill(8), format='%m%d%Y')
contribs_by_indivs.head()

,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,ZIP_CODE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID
15,C00428110,N,M2,P,201902049145460549,15,IND,"MATTHIES, KEVIN",WICHITA,KS,67210,SPIRIT AEROSYSTEMS INC,SENIOR VP/GENERAL MANAGER,2019-01-10,208,NaN,B000235S000077L11A1,1314300,NaN,NaN,4022620191644153387
16,C00428110,N,M2,P,201902049145460548,15,IND,"GENTILE, THOMAS C",WICHITA,KS,67210,SPIRIT AEROSYSTEMS INC,PRESIDENT & CEO,2019-01-10,416,NaN,B000235S000036L11A1,1314300,NaN,NaN,4022620191644153384
17,C00428110,N,M2,P,201902049145460548,15,IND,"HAWKINS, DUANE F",WICHITA,KS,67210,SPIRIT AEROSYSTEMS INC,SENIOR VP/GENERAL MANAGER,2019-01-10,250,NaN,B000235S000045L11A1,1314300,NaN,NaN,4022620191644153385
18,C00428110,N,M2,P,201902049145460548,15,IND,"KONDROTIS, KRISSTIE A",WICHITA,KS,67210,SPIRIT AEROSYSTEMS INC,SVP BUSINESS DEVELOPMENT & REGIO,2019-01-10,250,NaN,B000235S000062L11A1,1314300,NaN,NaN,4022620191644153386
19,C00428110,N,M2,P,201902049145460549,15,IND,"PILLA, JOHN ANTHONY",WICHITA,KS,67210,SPIRIT AEROSYSTEMS INC,SVP ENGINEERING AND CTO,2019-01-10,250,NaN,B000235S000090L11A1,1314300,NaN,NaN,4022620191644153388


In [47]:
contribs_by_indivs = contribs_by_indivs[(contribs_by_indivs['TRANSACTION_DT'] >= '2018-01-01') & (contribs_by_indivs['TRANSACTION_DT'] < '2021-01-01')]
contribs_by_indivs.head()

,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,ZIP_CODE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID
15,C00428110,N,M2,P,201902049145460549,15,IND,"MATTHIES, KEVIN",WICHITA,KS,67210,SPIRIT AEROSYSTEMS INC,SENIOR VP/GENERAL MANAGER,2019-01-10,208,NaN,B000235S000077L11A1,1314300,NaN,NaN,4022620191644153387
16,C00428110,N,M2,P,201902049145460548,15,IND,"GENTILE, THOMAS C",WICHITA,KS,67210,SPIRIT AEROSYSTEMS INC,PRESIDENT & CEO,2019-01-10,416,NaN,B000235S000036L11A1,1314300,NaN,NaN,4022620191644153384
17,C00428110,N,M2,P,201902049145460548,15,IND,"HAWKINS, DUANE F",WICHITA,KS,67210,SPIRIT AEROSYSTEMS INC,SENIOR VP/GENERAL MANAGER,2019-01-10,250,NaN,B000235S000045L11A1,1314300,NaN,NaN,4022620191644153385
18,C00428110,N,M2,P,201902049145460548,15,IND,"KONDROTIS, KRISSTIE A",WICHITA,KS,67210,SPIRIT AEROSYSTEMS INC,SVP BUSINESS DEVELOPMENT & REGIO,2019-01-10,250,NaN,B000235S000062L11A1,1314300,NaN,NaN,4022620191644153386
19,C00428110,N,M2,P,201902049145460549,15,IND,"PILLA, JOHN ANTHONY",WICHITA,KS,67210,SPIRIT AEROSYSTEMS INC,SVP ENGINEERING AND CTO,2019-01-10,250,NaN,B000235S000090L11A1,1314300,NaN,NaN,4022620191644153388


In [48]:
contribs_by_indivs = contribs_by_indivs.merge(cmte_cand_mapping[['CMTE_ID', 'CAND_NAME']], on='CMTE_ID')
#contribs_by_indivs = contribs_by_indivs.merge(candidates[['CAND_ID', 'CAND_NAME']], on='CAND_ID')
contribs_by_indivs.head()

,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,ZIP_CODE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID,CAND_NAME
0,C00428110,N,M2,P,201902049145460549,15,IND,"MATTHIES, KEVIN",WICHITA,KS,67210,SPIRIT AEROSYSTEMS INC,SENIOR VP/GENERAL MANAGER,2019-01-10,208,NaN,B000235S000077L11A1,1314300,NaN,NaN,4022620191644153387,"TILLIS, THOM R. SEN."
1,C00428110,N,M2,P,201902049145460548,15,IND,"GENTILE, THOMAS C",WICHITA,KS,67210,SPIRIT AEROSYSTEMS INC,PRESIDENT & CEO,2019-01-10,416,NaN,B000235S000036L11A1,1314300,NaN,NaN,4022620191644153384,"TILLIS, THOM R. SEN."
2,C00428110,N,M2,P,201902049145460548,15,IND,"HAWKINS, DUANE F",WICHITA,KS,67210,SPIRIT AEROSYSTEMS INC,SENIOR VP/GENERAL MANAGER,2019-01-10,250,NaN,B000235S000045L11A1,1314300,NaN,NaN,4022620191644153385,"TILLIS, THOM R. SEN."
3,C00428110,N,M2,P,201902049145460548,15,IND,"KONDROTIS, KRISSTIE A",WICHITA,KS,67210,SPIRIT AEROSYSTEMS INC,SVP BUSINESS DEVELOPMENT & REGIO,2019-01-10,250,NaN,B000235S000062L11A1,1314300,NaN,NaN,4022620191644153386,"TILLIS, THOM R. SEN."
4,C00428110,N,M2,P,201902049145460549,15,IND,"PILLA, JOHN ANTHONY",WICHITA,KS,67210,SPIRIT AEROSYSTEMS INC,SVP ENGINEERING AND CTO,2019-01-10,250,NaN,B000235S000090L11A1,1314300,NaN,NaN,4022620191644153388,"TILLIS, THOM R. SEN."


In [49]:
from IPython.display import display
for cand_name in candidates['CAND_NAME'].unique():
    print("-"*60)
    print(f"Most individual contributions for {cand_name}")
    print("-"*60)
    df = contribs_by_indivs[(contribs_by_indivs['CAND_NAME'] == cand_name)]
    result = pd.DataFrame(df[['NAME', 'TRANSACTION_AMT']].groupby(['NAME']).agg({'TRANSACTION_AMT': 'sum'})['TRANSACTION_AMT'].nlargest(20)).reset_index()
    result['TRANSACTION_AMT'] = result['TRANSACTION_AMT'].map(lambda x: render_human_format(x))
    display(result)

------------------------------------------------------------
Most individual contributions for CUNNINGHAM, CAL
------------------------------------------------------------


,NAME,TRANSACTION_AMT
0,SMP,7.85M
1,"ROSENTHAL, RICHARD",2.51M
2,"MANDEL, SUSAN",1.58M
3,"SIMON, DEBORAH",1.11M
4,"DELANEY, MARY QUINN",1.01M
5,"INDIVISIBLE PROJECT, INC.",1.00M
6,CHC BOLD PAC,803.64K
7,"SIMON, DEBORAH J",600.00K
8,"SOROS, GEORGE",512.00K
9,"O'BRIEN, PAULA",505.00K


------------------------------------------------------------
Most individual contributions for TILLIS, THOM R. SEN.
------------------------------------------------------------


,NAME,TRANSACTION_AMT
0,"SCHWARZMAN, STEPHEN A.",10.01M
1,"MELLON, TIMOTHY",10.00M
2,KOCH INDUSTRIES INC.,7.00M
3,FREEDOM PARTNERS ACTION FUND INC.,6.48M
4,"MARCUS, BERNARD",4.00M
5,"SCHWAB, CHARLES R.",3.50M
6,"DUNCAN, JAN",2.99M
7,SENATE LEADERSHIP FUND,2.95M
8,"STEPHENS, WARREN A.",2.75M
9,"SCHWAB, HELEN O'NEILL",2.50M


In [50]:
from IPython.display import display
indiv_contribs_cmte = contribs_by_indivs.merge(committee_master[['CMTE_ID', 'CMTE_NM']], on='CMTE_ID')
for cand_name in candidates['CAND_NAME'].unique():
    print("-"*80)
    print(f"Committees that received the most individual contributions for {cand_name}")
    print("-"*80)
    df = indiv_contribs_cmte[(indiv_contribs_cmte['CAND_NAME'] == cand_name)]
    result = pd.DataFrame(df[['CMTE_NM', 'TRANSACTION_AMT']].groupby(['CMTE_NM']).agg({
        'TRANSACTION_AMT': 'sum'})['TRANSACTION_AMT'].nlargest(10)).reset_index()
    result['TRANSACTION_AMT'] = result['TRANSACTION_AMT'].map(lambda x: render_human_format(x))
    display(result)

--------------------------------------------------------------------------------
Committees that received the most individual contributions for CUNNINGHAM, CAL
--------------------------------------------------------------------------------


,CMTE_NM,TRANSACTION_AMT
0,DSCC,58.86M
1,VOTEVETS,12.93M
2,PLANNED PARENTHOOD VOTES,10.02M
3,END CITIZENS UNITED,8.26M
4,MCCREADY FOR CONGRESS,4.22M
5,JSTREETPAC,3.05M
6,INTERNATIONAL ASSOCIATION OF FIREFIGHTERS INTE...,2.21M
7,CHC BOLD PAC,2.18M
8,INDIVISIBLE ACTION,2.17M
9,LEAGUE OF CONSERVATION VOTERS ACTION FUND,1.66M


--------------------------------------------------------------------------------
Committees that received the most individual contributions for TILLIS, THOM R. SEN.
--------------------------------------------------------------------------------


,CMTE_NM,TRANSACTION_AMT
0,SENATE LEADERSHIP FUND,63.36M
1,NRSC,54.02M
2,"AMERICANS FOR PROSPERITY ACTION, INC.(AFP ACTION)",24.60M
3,HONEYWELL INTERNATIONAL POLITICAL ACTION COMMI...,4.52M
4,WOMEN SPEAK OUT PAC,4.42M
5,NATIONAL ASSOCIATION OF REALTORS POLITICAL ACT...,4.00M
6,THE BOEING COMPANY POLITICAL ACTION COMMITTEE,3.57M
7,GOPAC ELECTION FUND,3.34M
8,DELOITTE POLITICAL ACTION COMMITTEE,3.16M
9,PRICEWATERHOUSECOOPERS POLITICAL ACTION COMMIT...,3.05M


In [51]:
contribs_by_indivs.loc[contribs_by_indivs['EMPLOYER'] == 'SELF', 'EMPLOYER'] = 'SELF-EMPLOYED'
contribs_by_indivs.loc[contribs_by_indivs['EMPLOYER'] == 'SELF EMPLOYED', 'EMPLOYER'] = 'SELF-EMPLOYED'
contribs_by_indivs.loc[contribs_by_indivs['EMPLOYER'] == 'NOT EMPLOYED', 'EMPLOYER'] = 'UNEMPLOYED'
contribs_by_indivs

,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,ZIP_CODE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID,CAND_NAME
0,C00428110,N,M2,P,201902049145460549,15,IND,"MATTHIES, KEVIN",WICHITA,KS,67210,SPIRIT AEROSYSTEMS INC,SENIOR VP/GENERAL MANAGER,2019-01-10,208,NaN,B000235S000077L11A1,1314300,NaN,NaN,4022620191644153387,"TILLIS, THOM R. SEN."
1,C00428110,N,M2,P,201902049145460548,15,IND,"GENTILE, THOMAS C",WICHITA,KS,67210,SPIRIT AEROSYSTEMS INC,PRESIDENT & CEO,2019-01-10,416,NaN,B000235S000036L11A1,1314300,NaN,NaN,4022620191644153384,"TILLIS, THOM R. SEN."
2,C00428110,N,M2,P,201902049145460548,15,IND,"HAWKINS, DUANE F",WICHITA,KS,67210,SPIRIT AEROSYSTEMS INC,SENIOR VP/GENERAL MANAGER,2019-01-10,250,NaN,B000235S000045L11A1,1314300,NaN,NaN,4022620191644153385,"TILLIS, THOM R. SEN."
3,C00428110,N,M2,P,201902049145460548,15,IND,"KONDROTIS, KRISSTIE A",WICHITA,KS,67210,SPIRIT AEROSYSTEMS INC,SVP BUSINESS DEVELOPMENT & REGIO,2019-01-10,250,NaN,B000235S000062L11A1,1314300,NaN,NaN,4022620191644153386,"TILLIS, THOM R. SEN."
4,C00428110,N,M2,P,201902049145460549,15,IND,"PILLA, JOHN ANTHONY",WICHITA,KS,67210,SPIRIT AEROSYSTEMS INC,SVP ENGINEERING AND CTO,2019-01-10,250,NaN,B000235S000090L11A1,1314300,NaN,NaN,4022620191644153388,"TILLIS, THOM R. SEN."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2714376,C00659508,N,Q1,P,202004139216653073,15,IND,"TABOR, JANE Y",FORT WALTON BEACH,FL,32547,NONE,NONE,2020-02-02,5000,NaN,SA11AI.4186,1395525,NaN,NaN,4041320201735609726,"TILLIS, THOM R. SEN."
2714377,C00659508,N,Q1,P,202004139216653073,15,IND,"YOUNG, DAVID B",CHAPEL HILL,NC,27517,THE CENTER FOR INT. ED. INC.,CEO,2020-02-10,5000,NaN,SA11AI.4185,1395525,NaN,NaN,4041320201735609728,"TILLIS, THOM R. SEN."
2714378,C00659508,N,Q1,P,202004139216653074,15,IND,"YOUNG, PHYLLIS",WEST END,NC,27376,NONE,NONE,2020-02-15,5000,NaN,SA11AI.4184,1395525,NaN,NaN,4041320201735609729,"TILLIS, THOM R. SEN."
2714379,C00736751,N,12P,P,202002209187234958,10,PAC,SENATE LEADERSHIP FUND,WARRENTON,VA,20186,NaN,NaN,2020-01-31,2450000,NaN,SA11C.4101,1385286,NaN,NaN,4022320201693922801,"TILLIS, THOM R. SEN."


In [52]:
top_employers = contribs_by_indivs['EMPLOYER'].value_counts()[:10].index
by_employer = contribs_by_indivs[contribs_by_indivs['EMPLOYER'].isin(top_employers)].groupby(['CAND_NAME', 'EMPLOYER']).agg({'TRANSACTION_AMT': 'sum'}).reset_index()
by_employer_pivot = by_employer.pivot_table(values='TRANSACTION_AMT', index='EMPLOYER', columns='CAND_NAME')
by_employer_pivot.iplot(kind='bar', colors=color_map, title='Total Individual contributions by Employer')

In [53]:
top_occupations = set(contribs_by_indivs['OCCUPATION'].value_counts()[:13].index) - set(['RETIRED', 'SELF-EMPLOYED', 'NOT EMPLOYED'])
by_occupation = contribs_by_indivs[contribs_by_indivs['OCCUPATION'].isin(top_occupations)].groupby(['CAND_NAME', 'OCCUPATION']).agg({'TRANSACTION_AMT': 'sum'}).reset_index()
by_occupation_pivot = by_occupation.pivot_table(values='TRANSACTION_AMT', index='OCCUPATION', columns='CAND_NAME')
by_occupation_pivot.iplot(kind='bar', colors=color_map, title='Total Individual contributions by Occupation')

In [54]:
by_date = contribs_by_indivs.groupby(['CAND_NAME', 'TRANSACTION_DT']).agg({'TRANSACTION_AMT': 'sum'})
by_date_cumsum = by_date.groupby(level=0).cumsum().reset_index().pivot_table(values='TRANSACTION_AMT', index='TRANSACTION_DT', columns='CAND_NAME')
by_date_cumsum.iplot(kind='line', colors=color_map, title='Total individual contributions over time')

In [55]:
contribs_by_indivs['in_state'] = np.where(contribs_by_indivs['STATE'] == state, 'in-state', 'out-of-state')
by_in_state = contribs_by_indivs.groupby(['CAND_NAME', 'in_state']).agg({'TRANSACTION_AMT': 'sum'}).reset_index()
by_in_state.pivot_table(values='TRANSACTION_AMT', index='in_state', columns='CAND_NAME').iplot(kind='bar', colors=color_map, title='In-state vs Out-of-state total individual contributions')

In [56]:
by_state_and_cand = contribs_by_indivs.groupby(['STATE', 'CAND_NAME']).agg({'TRANSACTION_AMT': 'sum'}).reset_index()
by_state_and_cand.head()

,STATE,CAND_NAME,TRANSACTION_AMT
0,AA,"CUNNINGHAM, CAL",11568
1,AA,"TILLIS, THOM R. SEN.",40
2,AE,"CUNNINGHAM, CAL",10412
3,AE,"TILLIS, THOM R. SEN.",4139
4,AK,"CUNNINGHAM, CAL",156673


In [57]:
by_state = by_state_and_cand.pivot_table(values='TRANSACTION_AMT', index='STATE', columns='CAND_NAME').reset_index()    
cand_cols = set(by_state.columns) - {'STATE'}
by_state.loc[:, cand_cols] = by_state[cand_cols].fillna(0.0)
by_state.head()

CAND_NAME,STATE,"CUNNINGHAM, CAL","TILLIS, THOM R. SEN."
0,AA,11568.0,40.0
1,AE,10412.0,4139.0
2,AK,156673.0,541903.0
3,AL,378135.0,4080413.0
4,AP,1244.0,2591.0


In [58]:
if democrat not in by_state.columns:
    by_state[democrat] = 0.0
if republican not in by_state.columns:
    by_state[republican] = 0.0

In [59]:
by_state['Total'] = by_state[democrat] + by_state[republican]
by_state['Republican_versus_Total_Ratio'] = by_state[republican]/by_state['Total']
by_state.head()

CAND_NAME,STATE,"CUNNINGHAM, CAL","TILLIS, THOM R. SEN.",Total,Republican_versus_Total_Ratio
0,AA,11568.0,40.0,11608.0,0.003446
1,AE,10412.0,4139.0,14551.0,0.284448
2,AK,156673.0,541903.0,698576.0,0.775725
3,AL,378135.0,4080413.0,4458548.0,0.915189
4,AP,1244.0,2591.0,3835.0,0.675619


In [60]:
import plotly.express as px
fig = px.choropleth(by_state, locations="STATE", color="Republican_versus_Total_Ratio", hover_name="STATE", 
                    hover_data=[democrat, republican], color_continuous_scale='bluered',
                    locationmode='USA-states', scope='usa', 
                    title='Total individual contributions for Republican vs Democrat by state',
                    range_color=[0, 1])
fig.show()

In [61]:
contribs_by_indivs['YEAR_MONTH'] = contribs_by_indivs['TRANSACTION_DT'].dt.to_period('M')
contribs_by_indivs.head()

,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,ZIP_CODE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID,CAND_NAME,in_state,YEAR_MONTH
0,C00428110,N,M2,P,201902049145460549,15,IND,"MATTHIES, KEVIN",WICHITA,KS,67210,SPIRIT AEROSYSTEMS INC,SENIOR VP/GENERAL MANAGER,2019-01-10,208,NaN,B000235S000077L11A1,1314300,NaN,NaN,4022620191644153387,"TILLIS, THOM R. SEN.",out-of-state,2019-01
1,C00428110,N,M2,P,201902049145460548,15,IND,"GENTILE, THOMAS C",WICHITA,KS,67210,SPIRIT AEROSYSTEMS INC,PRESIDENT & CEO,2019-01-10,416,NaN,B000235S000036L11A1,1314300,NaN,NaN,4022620191644153384,"TILLIS, THOM R. SEN.",out-of-state,2019-01
2,C00428110,N,M2,P,201902049145460548,15,IND,"HAWKINS, DUANE F",WICHITA,KS,67210,SPIRIT AEROSYSTEMS INC,SENIOR VP/GENERAL MANAGER,2019-01-10,250,NaN,B000235S000045L11A1,1314300,NaN,NaN,4022620191644153385,"TILLIS, THOM R. SEN.",out-of-state,2019-01
3,C00428110,N,M2,P,201902049145460548,15,IND,"KONDROTIS, KRISSTIE A",WICHITA,KS,67210,SPIRIT AEROSYSTEMS INC,SVP BUSINESS DEVELOPMENT & REGIO,2019-01-10,250,NaN,B000235S000062L11A1,1314300,NaN,NaN,4022620191644153386,"TILLIS, THOM R. SEN.",out-of-state,2019-01
4,C00428110,N,M2,P,201902049145460549,15,IND,"PILLA, JOHN ANTHONY",WICHITA,KS,67210,SPIRIT AEROSYSTEMS INC,SVP ENGINEERING AND CTO,2019-01-10,250,NaN,B000235S000090L11A1,1314300,NaN,NaN,4022620191644153388,"TILLIS, THOM R. SEN.",out-of-state,2019-01


In [62]:
by_date_state_and_cand = contribs_by_indivs.groupby(['YEAR_MONTH', 'STATE', 'CAND_NAME']).agg({'TRANSACTION_AMT': 'sum'}).reset_index()
by_date_state_and_cand.head()

,YEAR_MONTH,STATE,CAND_NAME,TRANSACTION_AMT
0,2018-10,CA,"TILLIS, THOM R. SEN.",800
1,2018-10,NV,"TILLIS, THOM R. SEN.",450
2,2018-11,AR,"TILLIS, THOM R. SEN.",156800
3,2018-11,AZ,"TILLIS, THOM R. SEN.",7300
4,2018-11,FL,"TILLIS, THOM R. SEN.",243400


In [63]:
dates = by_date_state_and_cand['YEAR_MONTH'].unique()
states = by_date_state_and_cand['STATE'].unique()
cands = by_date_state_and_cand['CAND_NAME'].unique()
cands

array(['TILLIS, THOM R. SEN.', 'CUNNINGHAM, CAL'], dtype=object)

In [64]:
by_date_state_and_cand_skel = pd.DataFrame([{'YEAR_MONTH': date, 'STATE': state, 'CAND_NAME': cand} for date in dates for state in states for cand in cands])
by_date_state_and_cand_skel['TRANSACTION_AMT'] = 0.0
by_date_state_and_cand_skel.head()

,YEAR_MONTH,STATE,CAND_NAME,TRANSACTION_AMT
0,2018-10,CA,"TILLIS, THOM R. SEN.",0.0
1,2018-10,CA,"CUNNINGHAM, CAL",0.0
2,2018-10,NV,"TILLIS, THOM R. SEN.",0.0
3,2018-10,NV,"CUNNINGHAM, CAL",0.0
4,2018-10,AR,"TILLIS, THOM R. SEN.",0.0


In [65]:
by_date_state_and_cand = by_date_state_and_cand_skel.merge(by_date_state_and_cand, how='left', on=['YEAR_MONTH', 'STATE', 'CAND_NAME'])
by_date_state_and_cand['TRANSACTION_AMT'] = np.where(by_date_state_and_cand['TRANSACTION_AMT_y'].isnull(), by_date_state_and_cand['TRANSACTION_AMT_x'], by_date_state_and_cand['TRANSACTION_AMT_y'])
by_date_state_and_cand = by_date_state_and_cand.drop('TRANSACTION_AMT_x', axis=1).drop('TRANSACTION_AMT_y', axis=1)
by_date_state_and_cand.head()

,YEAR_MONTH,STATE,CAND_NAME,TRANSACTION_AMT
0,2018-10,CA,"TILLIS, THOM R. SEN.",800.0
1,2018-10,CA,"CUNNINGHAM, CAL",0.0
2,2018-10,NV,"TILLIS, THOM R. SEN.",450.0
3,2018-10,NV,"CUNNINGHAM, CAL",0.0
4,2018-10,AR,"TILLIS, THOM R. SEN.",0.0


In [66]:
by_date_state_and_cand = by_date_state_and_cand.sort_values('YEAR_MONTH').set_index(['YEAR_MONTH', 'STATE', 'CAND_NAME']).groupby(level=[1,2]).cumsum().reset_index()
by_date_state_and_cand.head()

,YEAR_MONTH,STATE,CAND_NAME,TRANSACTION_AMT
0,2018-10,CA,"TILLIS, THOM R. SEN.",800.0
1,2018-10,OH,"CUNNINGHAM, CAL",0.0
2,2018-10,OH,"TILLIS, THOM R. SEN.",0.0
3,2018-10,NM,"CUNNINGHAM, CAL",0.0
4,2018-10,NM,"TILLIS, THOM R. SEN.",0.0


In [67]:
by_date_state = by_date_state_and_cand.pivot_table(values='TRANSACTION_AMT', index=['YEAR_MONTH', 'STATE'], columns='CAND_NAME').reset_index()    
cand_cols = set(by_date_state.columns) - {'YEAR_MONTH', 'STATE'}
by_date_state.loc[:, cand_cols] = by_date_state[cand_cols].fillna(0.0)
by_date_state.head()

CAND_NAME,YEAR_MONTH,STATE,"CUNNINGHAM, CAL","TILLIS, THOM R. SEN."
0,2018-10,AA,0.0,0.0
1,2018-10,AE,0.0,0.0
2,2018-10,AK,0.0,0.0
3,2018-10,AL,0.0,0.0
4,2018-10,AP,0.0,0.0


In [68]:
if democrat not in by_date_state.columns:
    by_date_state[democrat] = 0.0
if republican not in by_date_state.columns:
    by_date_state[republican] = 0.0

In [69]:
by_date_state['Total'] = by_date_state[democrat] + by_date_state[republican]
by_date_state['RepublicanVsTotalRatio'] = by_date_state[republican]/by_date_state['Total']
by_date_state = by_date_state.sort_values('YEAR_MONTH')
by_date_state['YEAR_MONTH'] = by_date_state['YEAR_MONTH'].astype('str')
by_date_state.head()

CAND_NAME,YEAR_MONTH,STATE,"CUNNINGHAM, CAL","TILLIS, THOM R. SEN.",Total,RepublicanVsTotalRatio
0,2018-10,AA,0.0,0.0,0.0,NaN
32,2018-10,MT,0.0,0.0,0.0,NaN
33,2018-10,NC,0.0,0.0,0.0,NaN
34,2018-10,ND,0.0,0.0,0.0,NaN
35,2018-10,NE,0.0,0.0,0.0,NaN
